In [1]:
import dash
from dash import dcc, html
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from dash.dependencies import Input, Output, State
import requests
from datetime import datetime, timedelta
from collections import Counter
import json
import os

# Ініціалізація додатку Dash з покращеними метаданими
app = dash.Dash(
    __name__,
    meta_tags=[
        {"name": "viewport", "content": "width=device-width, initial-scale=1.0"}
    ],
    title="Інформаційно-аналітична система НГУ",
    suppress_callback_exceptions=True,
    external_stylesheets=[
        "https://cdn.jsdelivr.net/npm/bootstrap@5.1.3/dist/css/bootstrap.min.css",
        "https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.0.0-beta3/css/all.min.css",
        "https://cdnjs.cloudflare.com/ajax/libs/animate.css/4.1.1/animate.min.css"
    ]
)

# Покращена кольорова схема з національними кольорами України
colors = {
    'visualization': '#E3F2FD',  # світло-блакитний
    'threats': '#FFF3E0',        # світло-оранжевий
    'resources': '#E8F5E9',      # світло-зелений
    'forecasting': '#F3E5F5',    # світло-фіолетовий
    'background': '#FAFAFA',     # світло-сірий фон
    'text': '#37474F',           # темно-сірий текст
    'accent': '#0057b7',         # синій (національний колір України)
    'accent2': '#ffd700',        # жовтий (національний колір України)
    'warning': '#FF9800',        # оранжевий для попереджень
    'danger': '#F44336',         # червоний для небезпеки
    'success': '#4CAF50'         # зелений для успіху
}

# Базовий URL для API-ендпоінтів
API_BASE_URL = "http://localhost:5000/api"

# Функції для отримання даних з API
def get_dashboard_statistics():
    try:
        response = requests.get(f"{API_BASE_URL}/dashboard/statistics")
        return response.json()
    except Exception as e:
        print(f"Помилка при отриманні статистики: {e}")
        # Повертаємо тестові дані у випадку помилки
        return {
            'units_count': 25,
            'resources_count': 150,
            'incidents_count': 42,
            'critical_situations_count': 3
        }

def get_recent_incidents():
    try:
        response = requests.get(f"{API_BASE_URL}/incidents/recent")
        return response.json()
    except Exception as e:
        print(f"Помилка при отриманні інцидентів: {e}")
        # Повертаємо тестові дані у випадку помилки
        return [
            {
                'id': 1,
                'type': 'Порушення громадського порядку',
                'subtype': 'Масові заворушення',
                'location': 'Київ',
                'timestamp': datetime.now().isoformat(),
                'severity': 'Високий',
                'status': 'Активний'
            }
        ]

def get_threat_level_history():
    try:
        response = requests.get(f"{API_BASE_URL}/analytics/threat-level-history")
        return response.json()
    except Exception as e:
        print(f"Помилка при отриманні історії рівня загрози: {e}")
        # Повертаємо тестові дані у випадку помилки
        dates = [(datetime.now() - timedelta(days=i)).strftime('%d.%m.%Y') for i in range(30, 0, -1)]
        return {
            'dates': dates,
            'threat_levels': [np.random.randint(30, 80) for _ in range(30)]
        }

def get_resources_distribution():
    try:
        response = requests.get(f"{API_BASE_URL}/resources/distribution")
        return response.json()
    except Exception as e:
        print(f"Помилка при отриманні розподілу ресурсів: {e}")
        # Повертаємо тестові дані у випадку помилки
        resources = [
            'Військове обладнання', 'Гуманітарна допомога', 'Енергетичні ресурси',
            'Медичні засоби', 'Системи зв\'язку', 'Транспорт', 'Продовольство'
        ]
        return {
            'resources': [
                {'id': i+1, 'name': res, 'type': res.split()[0], 'usage_percentage': np.random.randint(50, 95)}
                for i, res in enumerate(resources)
            ],
            'by_type': {
                res.split()[0]: {'count': np.random.randint(10, 50), 'percentage': np.random.randint(10, 30)}
                for res in resources
            },
            'by_status': {
                'Доступний': {'count': 120, 'percentage': 80},
                'В ремонті': {'count': 15, 'percentage': 10},
                'Недоступний': {'count': 15, 'percentage': 10}
            }
        }

def get_map_data():
    try:
        response = requests.get(f"{API_BASE_URL}/situations/map-data")
        return response.json()
    except Exception as e:
        print(f"Помилка при отриманні даних для карти: {e}")
        # Повертаємо тестові дані у випадку помилки
        regions = [
            'Київська', 'Львівська', 'Харківська', 'Одеська', 'Дніпропетровська',
            'Донецька', 'Луганська', 'Запорізька', 'Херсонська'
        ]
        statuses = ['Нормальна', 'Напружена', 'Критична']
        return [
            {
                'id': i+1,
                'location': region,
                'region': region,
                'status': np.random.choice(statuses),
                'threat_level': np.random.randint(1, 10),
                'timestamp': datetime.now().isoformat(),
                'coordinates': f"{np.random.uniform(44.0, 52.0):.6f}, {np.random.uniform(22.0, 40.0):.6f}"
            }
            for i, region in enumerate(regions)
        ]

def get_ai_predictions():
    try:
        response = requests.get(f"{API_BASE_URL}/analytics/predictions")
        return response.json()
    except Exception as e:
        print(f"Помилка при отриманні прогнозів AI: {e}")
        # Повертаємо тестові дані у випадку помилки
        return [
            {
                'id': 1,
                'type': 'Загроза безпеці',
                'region': 'Харківська',
                'location': 'Харків',
                'value': 8.5,
                'confidence': 0.85,
                'description': 'Прогнозується підвищення рівня загрози в регіоні',
                'timestamp': datetime.now().isoformat()
            }
        ]

# Функції для створення графіків
def create_ukraine_map(map_data=None):
    if map_data is None:
        map_data = get_map_data()
    
    # Створюємо базову карту
    fig = go.Figure()
    
    # Додаємо маркери для кожної ситуації
    for situation in map_data:
        # Парсимо координати
        try:
            lat, lon = map(float, situation['coordinates'].split(','))
        except:
            continue
        
        # Визначаємо колір маркера залежно від статусу
        if situation['status'] == 'Критична':
            marker_color = colors['danger']
        elif situation['status'] == 'Напружена':
            marker_color = colors['warning']
        else:
            marker_color = colors['success']
        
        # Додаємо маркер
        fig.add_trace(go.Scattergeo(
            lon=[lon],
            lat=[lat],
            text=situation['location'],
            mode='markers',
            marker=dict(
                size=10,
                color=marker_color,
                line=dict(width=1, color='white')
            ),
            hoverinfo='text',
            hovertext=f"<b>{situation['location']}</b><br>" +
                      f"Статус: {situation['status']}<br>" +
                      f"Рівень загрози: {situation['threat_level']}/10<br>" +
                      f"Оновлено: {situation['timestamp'].split('T')[0]}"
        ))
    
    # Налаштовуємо вигляд карти
    fig.update_geos(
        visible=False,
        resolution=50,
        scope="europe",
        showcountries=True,
        countrycolor="gray",
        showsubunits=True,
        subunitcolor="blue",
        center=dict(lon=31.1656, lat=48.3794),  # Центр України
        projection_scale=5
    )
    
    fig.update_layout(
        title={
            'text': 'Карта оперативної обстановки',
            'y': 0.95,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': {'size': 20, 'color': colors['text']}
        },
        margin={"r":0, "t":30, "l":0, "b":0},
        height=500,
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        geo=dict(
            bgcolor='rgba(0,0,0,0)'
        )
    )
    
    return fig

def create_threat_analysis():
    # Отримуємо дані про рівень загрози
    threat_data = get_threat_level_history()
    
    # Створюємо графік
    fig = go.Figure()
    
    # Додаємо лінію рівня загрози
    fig.add_trace(go.Scatter(
        x=threat_data['dates'],
        y=threat_data['threat_levels'],
        mode='lines+markers',
        name='Рівень загрози',
        line=dict(color=colors['accent'], width=3),
        marker=dict(size=8, color=colors['accent']),
        fill='tozeroy',
        fillcolor=f"rgba({int(colors['accent'][1:3], 16)}, {int(colors['accent'][3:5], 16)}, {int(colors['accent'][5:7], 16)}, 0.2)"
    ))
    
    # Додаємо зони рівнів загрози
    fig.add_shape(
        type="rect",
        x0=0,
        y0=70,
        x1=len(threat_data['dates']),
        y1=100,
        fillcolor="rgba(76, 175, 80, 0.1)",
        line=dict(width=0),
        layer="below"
    )
    
    fig.add_shape(
        type="rect",
        x0=0,
        y0=40,
        x1=len(threat_data['dates']),
        y1=70,
        fillcolor="rgba(255, 152, 0, 0.1)",
        line=dict(width=0),
        layer="below"
    )
    
    fig.add_shape(
        type="rect",
        x0=0,
        y0=0,
        x1=len(threat_data['dates']),
        y1=40,
        fillcolor="rgba(244, 67, 54, 0.1)",
        line=dict(width=0),
        layer="below"
    )
    
    # Додаємо анотації для зон
    fig.add_annotation(
        x=len(threat_data['dates']) - 1,
        y=85,
        text="Низький рівень загрози",
        showarrow=False,
        font=dict(size=10, color="rgba(76, 175, 80, 0.7)")
    )
    
    fig.add_annotation(
        x=len(threat_data['dates']) - 1,
        y=55,
        text="Середній рівень загрози",
        showarrow=False,
        font=dict(size=10, color="rgba(255, 152, 0, 0.7)")
    )
    
    fig.add_annotation(
        x=len(threat_data['dates']) - 1,
        y=20,
        text="Високий рівень загрози",
        showarrow=False,
        font=dict(size=10, color="rgba(244, 67, 54, 0.7)")
    )
    
    # Налаштовуємо вигляд графіка
    fig.update_layout(
        title={
            'text': 'Динаміка рівня загрози',
            'y': 0.95,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': {'size': 20, 'color': colors['text']}
        },
        xaxis=dict(
            title='Дата',
            tickangle=45,
            tickmode='array',
            tickvals=[i for i in range(0, len(threat_data['dates']), 5)],
            ticktext=[threat_data['dates'][i] for i in range(0, len(threat_data['dates']), 5)],
            gridcolor='rgba(230, 230, 230, 0.5)'
        ),
        yaxis=dict(
            title='Рівень загрози',
            range=[0, 100],
            gridcolor='rgba(230, 230, 230, 0.5)'
        ),
        margin=dict(l=40, r=40, t=50, b=40),
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        hovermode='x unified'
    )
    
    return fig

def create_resource_management():
    # Отримуємо дані про ресурси
    resource_data = get_resources_distribution()
    
    # Створюємо дані для графіка
    resource_types = list(resource_data['by_type'].keys())
    resource_counts = [data['count'] for data in resource_data['by_type'].values()]
    resource_percentages = [data['percentage'] for data in resource_data['by_type'].values()]
    
    # Створюємо графік розподілу ресурсів за типами
    fig = go.Figure()
    
    # Додаємо стовпчики для кожного типу ресурсів
    fig.add_trace(go.Bar(
        x=resource_types,
        y=resource_counts,
        marker_color=colors['accent'],
        text=resource_percentages,
        texttemplate='%{text}%',
        textposition='outside',
        hovertemplate='<b>%{x}</b><br>Кількість: %{y}<br>Відсоток: %{text}%<extra></extra>'
    ))
    
    # Налаштовуємо вигляд графіка
    fig.update_layout(
        title={
            'text': 'Розподіл ресурсів за типами',
            'y': 0.95,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': {'size': 20, 'color': colors['text']}
        },
        xaxis=dict(
            title='Тип ресурсу',
            tickangle=45,
            gridcolor='rgba(230, 230, 230, 0.5)'
        ),
        yaxis=dict(
            title='Кількість',
            gridcolor='rgba(230, 230, 230, 0.5)'
        ),
        margin=dict(l=40, r=40, t=50, b=40),
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)'
    )
    
    return fig

def create_resource_status_chart():
    # Отримуємо дані про ресурси
    resource_data = get_resources_distribution()
    
    # Створюємо дані для графіка
    status_labels = list(resource_data['by_status'].keys())
    status_values = [data['percentage'] for data in resource_data['by_status'].values()]
    
    # Визначаємо кольори для статусів
    status_colors = {
        'Доступний': colors['success'],
        'В ремонті': colors['warning'],
        'Недоступний': colors['danger']
    }
    
    # Створюємо графік розподілу ресурсів за статусами
    fig = go.Figure()
    
    # Додаємо кругову діаграму
    fig.add_trace(go.Pie(
        labels=status_labels,
        values=status_values,
        marker=dict(
            colors=[status_colors.get(status, colors['accent']) for status in status_labels],
            line=dict(color='white', width=2)
        ),
        textinfo='label+percent',
        insidetextorientation='radial',
        hole=0.4
    ))
    
    # Налаштовуємо вигляд графіка
    fig.update_layout(
        title={
            'text': 'Статус ресурсів',
            'y': 0.95,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': {'size': 20, 'color': colors['text']}
        },
        margin=dict(l=20, r=20, t=50, b=20),
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=-0.1,
            xanchor="center",
            x=0.5
        )
    )
    
    return fig

def create_incidents_chart():
    # Отримуємо дані про інциденти
    incidents = get_recent_incidents()
    
    # Групуємо інциденти за типами
    incident_types = {}
    for incident in incidents:
        incident_type = incident['type']
        if incident_type in incident_types:
            incident_types[incident_type] += 1
        else:
            incident_types[incident_type] = 1
    
    # Створюємо дані для графіка
    types = list(incident_types.keys())
    counts = list(incident_types.values())
    
    # Створюємо графік інцидентів
    fig = go.Figure()
    
    # Додаємо стовпчики для кожного типу інцидентів
    fig.add_trace(go.Bar(
        x=types,
        y=counts,
        marker_color=colors['warning'],
        text=counts,
        textposition='outside',
        hovertemplate='<b>%{x}</b><br>Кількість: %{y}<extra></extra>'
    ))
    
    # Налаштовуємо вигляд графіка
    fig.update_layout(
        title={
            'text': 'Розподіл інцидентів за типами',
            'y': 0.95,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': {'size': 20, 'color': colors['text']}
        },
        xaxis=dict(
            title='Тип інциденту',
            tickangle=45,
            gridcolor='rgba(230, 230, 230, 0.5)'
        ),
        yaxis=dict(
            title='Кількість',
            gridcolor='rgba(230, 230, 230, 0.5)'
        ),
        margin=dict(l=40, r=40, t=50, b=40),
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)'
    )
    
    return fig

# Макет додатку
app.layout = html.Div(style={'backgroundColor': colors['background'], 'minHeight': '100vh'}, children=[
    # Заголовок
    html.Div(className="container-fluid", children=[
        html.Div(className="row mb-4", children=[
            html.Div(className="col-12", children=[
                html.H1("ІНФОРМАЦІЙНО-АНАЛІТИЧНА СИСТЕМА НГУ", className="display-5 fw-bold text-center mt-4 animate__animated animate__fadeIn"),
                html.P("Інтерактивний дашборд для моніторингу та управління ресурсами Національної гвардії України", 
                       className="lead text-muted text-center animate__animated animate__fadeIn")
            ])
        ])
    ]),
    
    # Статистика
    html.Div(className="container-fluid", children=[
        html.Div(className="row g-4 mb-4", children=[
            # Кількість підрозділів
            html.Div(className="col-md-6 col-lg-3", children=[
                html.Div(className="card h-100 animate__animated animate__fadeInUp", children=[
                    html.Div(className="card-body text-center", children=[
                        html.Div(className="d-flex align-items-center justify-content-center mb-3", children=[
                            html.I(className="fas fa-users fa-3x text-primary me-3"),
                            html.H2(id="units-count", className="display-4 mb-0")
                        ]),
                        html.H3("Підрозділи", className="card-title h5 mb-2"),
                        html.P("Загальна кількість підрозділів НГУ", className="card-text text-muted")
                    ])
                ])
            ]),
            
            # Кількість ресурсів
            html.Div(className="col-md-6 col-lg-3", children=[
                html.Div(className="card h-100 animate__animated animate__fadeInUp", 
                         style={"animation-delay": "0.1s"}, children=[
                    html.Div(className="card-body text-center", children=[
                        html.Div(className="d-flex align-items-center justify-content-center mb-3", children=[
                            html.I(className="fas fa-boxes fa-3x text-success me-3"),
                            html.H2(id="resources-count", className="display-4 mb-0")
                        ]),
                        html.H3("Ресурси", className="card-title h5 mb-2"),
                        html.P("Доступні ресурси в системі", className="card-text text-muted")
                    ])
                ])
            ]),
            
            # Кількість інцидентів
            html.Div(className="col-md-6 col-lg-3", children=[
                html.Div(className="card h-100 animate__animated animate__fadeInUp", 
                         style={"animation-delay": "0.2s"}, children=[
                    html.Div(className="card-body text-center", children=[
                        html.Div(className="d-flex align-items-center justify-content-center mb-3", children=[
                            html.I(className="fas fa-exclamation-triangle fa-3x text-warning me-3"),
                            html.H2(id="incidents-count", className="display-4 mb-0")
                        ]),
                        html.H3("Інциденти", className="card-title h5 mb-2"),
                        html.P("Зареєстровані інциденти за 30 днів", className="card-text text-muted")
                    ])
                ])
            ]),
            
            # Критичні ситуації
            html.Div(className="col-md-6 col-lg-3", children=[
                html.Div(className="card h-100 animate__animated animate__fadeInUp", 
                         style={"animation-delay": "0.3s"}, children=[
                    html.Div(className="card-body text-center", children=[
                        html.Div(className="d-flex align-items-center justify-content-center mb-3", children=[
                            html.I(className="fas fa-radiation fa-3x text-danger me-3"),
                            html.H2(id="critical-count", className="display-4 mb-0")
                        ]),
                        html.H3("Критичні ситуації", className="card-title h5 mb-2"),
                        html.P("Активні критичні ситуації", className="card-text text-muted")
                    ])
                ])
            ])
        ])
    ]),
    
    # Основні графіки
    html.Div(className="container-fluid", children=[
        html.Div(className="row g-4 mb-4", children=[
            # Карта України
            html.Div(className="col-lg-8", children=[
                html.Div(className="card h-100 animate__animated animate__fadeIn", 
                         style={"animation-delay": "0.4s"}, children=[
                    html.Div(className="card-header d-flex justify-content-between align-items-center", children=[
                        html.H5("Карта оперативної обстановки", className="mb-0"),
                        html.Button("Оновити", id="update-map", className="btn btn-sm btn-outline-primary")
                    ]),
                    html.Div(className="card-body p-0", children=[
                        dcc.Graph(id="ukraine-map", figure=create_ukraine_map(), config={'displayModeBar': False})
                    ])
                ])
            ]),
            
            # Аналіз загроз
            html.Div(className="col-lg-4", children=[
                html.Div(className="card h-100 animate__animated animate__fadeIn", 
                         style={"animation-delay": "0.5s"}, children=[
                    html.Div(className="card-header", children=[
                        html.H5("Аналіз загроз", className="mb-0")
                    ]),
                    html.Div(className="card-body", children=[
                        dcc.Graph(id="threat-analysis", figure=create_threat_analysis(), config={'displayModeBar': False})
                    ])
                ])
            ])
        ]),
        
        # Другий ряд графіків
        html.Div(className="row g-4 mb-4", children=[
            # Управління ресурсами
            html.Div(className="col-lg-6", children=[
                html.Div(className="card h-100 animate__animated animate__fadeIn", 
                         style={"animation-delay": "0.6s"}, children=[
                    html.Div(className="card-header d-flex justify-content-between align-items-center", children=[
                        html.H5("Управління ресурсами", className="mb-0"),
                        html.Div(className="btn-group", children=[
                            html.Button("За типами", id="resource-type-btn", className="btn btn-sm btn-outline-primary active"),
                            html.Button("За статусом", id="resource-status-btn", className="btn btn-sm btn-outline-primary")
                        ])
                    ]),
                    html.Div(className="card-body", children=[
                        html.Div(id="resource-chart-container", children=[
                            dcc.Graph(id="resource-chart", figure=create_resource_management(), config={'displayModeBar': False})
                        ])
                    ])
                ])
            ]),
            
            # Інциденти
            html.Div(className="col-lg-6", children=[
                html.Div(className="card h-100 animate__animated animate__fadeIn", 
                         style={"animation-delay": "0.7s"}, children=[
                    html.Div(className="card-header", children=[
                        html.H5("Останні інциденти", className="mb-0")
                    ]),
                    html.Div(className="card-body", children=[
                        html.Div(id="incidents-table-container", style={"maxHeight": "400px", "overflow": "auto"}, children=[
                            html.Table(className="table table-hover", children=[
                                html.Thead(className="table-light", children=[
                                    html.Tr(children=[
                                        html.Th("Тип"),
                                        html.Th("Локація"),
                                        html.Th("Дата"),
                                        html.Th("Статус"),
                                        html.Th("Рівень")
                                    ])
                                ]),
                                html.Tbody(id="incidents-table-body")
                            ])
                        ])
                    ])
                ])
            ])
        ]),
        
        # Третій ряд - AI прогнози
        html.Div(className="row g-4 mb-4", children=[
            html.Div(className="col-12", children=[
                html.Div(className="card animate__animated animate__fadeIn", 
                         style={"animation-delay": "0.8s"}, children=[
                    html.Div(className="card-header d-flex justify-content-between align-items-center", children=[
                        html.H5("Прогнози штучного інтелекту", className="mb-0"),
                        html.Span(className="badge bg-primary", children=["AI"])
                    ]),
                    html.Div(className="card-body", children=[
                        html.Div(className="row", id="ai-predictions-container")
                    ])
                ])
            ])
        ])
    ]),
    
    # Інтервал для автоматичного оновлення даних
    dcc.Interval(
        id='interval-component',
        interval=60*1000,  # оновлення кожну хвилину
        n_intervals=0
    )
])

# Колбеки для оновлення даних
@app.callback(
    [
        Output("units-count", "children"),
        Output("resources-count", "children"),
        Output("incidents-count", "children"),
        Output("critical-count", "children")
    ],
    [Input("interval-component", "n_intervals")]
)
def update_statistics(n):
    """Оновлення статистичних даних"""
    stats = get_dashboard_statistics()
    return [
        stats['units_count'],
        stats['resources_count'],
        stats['incidents_count'],
        stats['critical_situations_count']
    ]

@app.callback(
    Output("ukraine-map", "figure"),
    [Input("update-map", "n_clicks"), Input("interval-component", "n_intervals")]
)
def update_map(n_clicks, n_intervals):
    """Оновлення карти"""
    return create_ukraine_map()

@app.callback(
    Output("threat-analysis", "figure"),
    [Input("interval-component", "n_intervals")]
)
def update_threat_analysis(n):
    """Оновлення аналізу загроз"""
    return create_threat_analysis()

@app.callback(
    Output("resource-chart-container", "children"),
    [Input("resource-type-btn", "n_clicks"), Input("resource-status-btn", "n_clicks")]
)
def update_resource_chart(type_clicks, status_clicks):
    """Перемикання між різними видами графіків ресурсів"""
    ctx = dash.callback_context
    if not ctx.triggered:
        # За замовчуванням показуємо розподіл за типами
        return [dcc.Graph(id="resource-chart", figure=create_resource_management(), config={'displayModeBar': False})]
    else:
        button_id = ctx.triggered[0]['prop_id'].split('.')[0]
        if button_id == "resource-status-btn":
            return [dcc.Graph(id="resource-chart", figure=create_resource_status_chart(), config={'displayModeBar': False})]
        else:
            return [dcc.Graph(id="resource-chart", figure=create_resource_management(), config={'displayModeBar': False})]

@app.callback(
    Output("incidents-table-body", "children"),
    [Input("interval-component", "n_intervals")]
)
def update_incidents_table(n):
    """Оновлення таблиці інцидентів"""
    incidents = get_recent_incidents()
    rows = []
    for incident in incidents:
        # Визначаємо клас для рядка залежно від рівня важливості
        severity_class = ""
        if incident['severity'] == 'Високий':
            severity_class = "table-danger"
        elif incident['severity'] == 'Середній':
            severity_class = "table-warning"
        
        # Форматуємо дату
        try:
            date_obj = datetime.fromisoformat(incident['timestamp'])
            formatted_date = date_obj.strftime("%d.%m.%Y %H:%M")
        except:
            formatted_date = incident['timestamp']
        
        # Додаємо рядок до таблиці
        rows.append(html.Tr(className=severity_class, children=[
            html.Td(f"{incident['type']} - {incident['subtype']}"),
            html.Td(incident['location']),
            html.Td(formatted_date),
            html.Td(incident['status']),
            html.Td(incident['severity'])
        ]))
    
    return rows

@app.callback(
    Output("ai-predictions-container", "children"),
    [Input("interval-component", "n_intervals")]
)
def update_ai_predictions(n):
    """Оновлення прогнозів AI"""
    predictions = get_ai_predictions()
    prediction_cards = []
    
    for prediction in predictions:
        # Визначаємо колір картки залежно від значення прогнозу
        if prediction['value'] > 7:
            card_class = "border-danger"
            header_class = "bg-danger text-white"
        elif prediction['value'] > 4:
            card_class = "border-warning"
            header_class = "bg-warning"
        else:
            card_class = "border-success"
            header_class = "bg-success text-white"
        
        # Форматуємо дату
        try:
            date_obj = datetime.fromisoformat(prediction['timestamp'])
            formatted_date = date_obj.strftime("%d.%m.%Y")
        except:
            formatted_date = prediction['timestamp']
        
        # Створюємо картку прогнозу
        prediction_cards.append(html.Div(className="col-md-4 mb-3", children=[
            html.Div(className=f"card h-100 {card_class}", children=[
                html.Div(className=f"card-header {header_class}", children=[
                    html.H6(prediction['type'], className="mb-0")
                ]),
                html.Div(className="card-body", children=[
                    html.H5(prediction['region'], className="card-title"),
                    html.P(prediction['description'], className="card-text"),
                    html.Div(className="d-flex justify-content-between align-items-center", children=[
                        html.Span(f"Значення: {prediction['value']}"),
                        html.Span(f"Достовірність: {int(prediction['confidence']*100)}%")
                    ])
                ]),
                html.Div(className="card-footer text-muted", children=[
                    f"Оновлено: {formatted_date}"
                ])
            ])
        ]))
    
    return prediction_cards

# НЕ ЗАПУСКАЄМО app.run_server() тут, оскільки інтеграція здійснюється через DispatcherMiddleware у Flask
if __name__ == '__main__':
    app.run_server(debug=True, port=8050)
    print("Сервер запущено. Відкрийте http://127.0.0.1:8050/ у вашому браузері.")

Помилка при отриманні даних для карти: HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /api/situations/map-data (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000017463AB7F10>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))
Помилка при отриманні історії рівня загрози: HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /api/analytics/threat-level-history (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000017462017790>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))
Помилка при отриманні розподілу ресурсів: HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /api/resources/distribution (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000017

Сервер запущено. Відкрийте http://127.0.0.1:8050/ у вашому браузері.


Помилка при отриманні даних для карти: HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /api/situations/map-data (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000174528B9710>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))
Помилка при отриманні статистики: HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /api/dashboard/statistics (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000017452655B50>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))
Помилка при отриманні історії рівня загрози: HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /api/analytics/threat-level-history (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000174528A80D0>

In [2]:

import dash
from dash import dcc, html
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from dash.dependencies import Input, Output
import requests
from datetime import datetime, timedelta
from collections import Counter
import uuid

# Ініціалізація додатку Dash
app = dash.Dash(__name__)

# Покращена кольорова схема
colors = {
    'visualization': '#E3F2FD',  # світло-блакитний
    'threats': '#FFF3E0',        # світло-оранжевий
    'resources': '#E8F5E9',      # світло-зелений
    'forecasting': '#F3E5F5',    # світло-фіолетовий
    'background': '#FAFAFA',     # світло-сірий фон
    'text': '#37474F',           # темно-сірий текст
    'accent': '#1976D2',         # акцентний синій
    'warning': '#FF9800',        # оранжевий для попереджень
    'danger': '#F44336',         # червоний для небезпеки
    'success': '#4CAF50'         # зелений для успіху
}

# Завантаження GeoJSON
# Використовуємо правильне джерело GeoJSON для України
ukraine_geojson_url = "https://raw.githubusercontent.com/vsapsai/ukraine_map_data/master/ukraine_geojson.json"
try:
    response = requests.get(ukraine_geojson_url)
    ukraine_geojson = response.json()
    print("GeoJSON успішно завантажено")
except Exception as e:
    print(f"Помилка при завантаженні GeoJSON: {e}")
    # Створюємо спрощений GeoJSON для України як запасний варіант
    # Координати областей України
    region_coords = {
        'Kyiv': (30.5, 50.4),
        'Lviv': (24.0, 49.8),
        'Kharkiv': (36.2, 49.9),
        'Odesa': (30.7, 46.5),
        'Dnipropetrovsk': (35.0, 48.5),
        'Donetsk': (37.8, 48.0),
        'Luhansk': (39.3, 48.9),
        'Zaporizhia': (35.4, 47.8),
        'Kherson': (33.5, 46.6),
        'Mykolaiv': (32.0, 47.0),
        'Vinnytsia': (28.5, 49.2),
        'Poltava': (34.6, 49.6),
        'Sumy': (34.8, 51.0),
        'Chernihiv': (31.3, 51.5),
        'Cherkasy': (32.0, 49.4),
        'Khmelnytskyi': (27.0, 49.4),
        'Rivne': (26.2, 50.6),
        'Volyn': (25.0, 51.0),
        'Ivano-Frankivsk': (24.7, 48.9),
        'Ternopil': (25.6, 49.6),
        'Zakarpattia': (22.3, 48.6),
        'Chernivtsi': (25.9, 48.3),
        'Zhytomyr': (28.7, 50.3),
        'Kirovohrad': (32.3, 48.5)
    }
    
    # Створюємо спрощений GeoJSON
    ukraine_geojson = {
        "type": "FeatureCollection",
        "features": [{
            "type": "Feature",
            "properties": {"shapeName": region},
            "geometry": {
                "type": "Polygon",
                "coordinates": [[[lon-1, lat-1], [lon+1, lat-1], [lon+1, lat+1], [lon-1, lat+1], [lon-1, lat-1]]]
            }
        } for region, (lon, lat) in region_coords.items()]
    }
    print("Створено запасний GeoJSON")

# Словник для відповідності назв регіонів
region_name_mapping = {
    'Київська': 'Kyiv',
    'Львівська': 'Lviv',
    'Харківська': 'Kharkiv',
    'Одеська': 'Odesa',
    'Дніпропетровська': 'Dnipropetrovsk',
    'Донецька': 'Donetsk',
    'Луганська': 'Luhansk',
    'Запорізька': 'Zaporizhia',
    'Херсонська': 'Kherson',
    'Миколаївська': 'Mykolaiv',
    'Вінницька': 'Vinnytsia',
    'Полтавська': 'Poltava',
    'Сумська': 'Sumy',
    'Чернігівська': 'Chernihiv',
    'Черкаська': 'Cherkasy',
    'Хмельницька': 'Khmelnytskyi',
    'Рівненська': 'Rivne',
    'Волинська': 'Volyn',
    'Івано-Франківська': 'Ivano-Frankivsk',
    'Тернопільська': 'Ternopil',
    'Закарпатська': 'Zakarpattia',
    'Чернівецька': 'Chernivtsi',
    'Житомирська': 'Zhytomyr',
    'Кіровоградська': 'Kirovohrad'
}

# Генерація тестових даних
def generate_test_data():
    regions = list(region_name_mapping.keys())
    security_levels = np.random.randint(10, 100, size=len(regions))
    high_risk_regions = ['Харківська', 'Донецька', 'Луганська', 'Запорізька', 'Херсонська', 'Сумська', 'Чернігівська']
    for region in high_risk_regions:
        if region in regions:
            idx = regions.index(region)
            security_levels[idx] = np.random.randint(10, 40)
    visualization_data = pd.DataFrame({'Область': regions, 'Рівень безпеки': security_levels})
    visualization_data['Region English'] = visualization_data['Область'].map(region_name_mapping)

    threat_types = [
        'Ракетні удари', 'Артилерійські обстріли', 'Безпілотники', 'Кібератаки',
        'Диверсійні групи', 'Інформаційні операції', 'Інфраструктурні атаки'
    ]
    risk_levels = np.random.randint(3, 10, size=len(threat_types))
    threat_categories = [
        'Військова загроза', 'Військова загроза', 'Військова загроза', 'Кібер загроза',
        'Фізична загроза', 'Інформаційна загроза', 'Критична інфраструктура'
    ]
    threats_data = pd.DataFrame({
        'Тип загрози': threat_types, 
        'Рівень ризику': risk_levels,
        'Категорія': threat_categories
    })

    resources = [
        'Військове обладнання', 'Гуманітарна допомога', 'Енергетичні ресурси',
        'Медичні засоби', 'Системи зв\'язку', 'Транспорт', 'Продовольство'
    ]
    usage = np.random.randint(50, 100, size=len(resources))
    resources_data = pd.DataFrame({'Ресурс': resources, 'Використання (%)': usage})

    dates = pd.date_range(start='2025-01-01', periods=30)
    forecasted = np.cumsum(np.random.normal(0, 2, size=len(dates))) + 50
    actual = forecasted[:20] + np.random.normal(0, 3, size=20)
    forecast_data = pd.DataFrame({
        'Дата': dates,
        'Прогноз': forecasted,
        'Фактично': np.append(actual, [None] * 10)
    })

    return visualization_data, threats_data, resources_data, forecast_data

viz_data, threats_data, resources_data, forecast_data = generate_test_data()

# Отримання даних ACLED
def fetch_acled_data():
    api_key = "your_api_key"  # Замініть на ваш ключ API
    email = "your_email"      # Замініть на вашу електронну пошту
    thirty_days_ago = (datetime.now() - timedelta(days=30)).strftime('%Y-%m-%d')
    
    # Для карти (усі події)
    try:
        url = f"https://api.acleddata.com/acled/read?key={api_key}&email={email}&country=Ukraine&event_date={thirty_days_ago}&event_date_where=>&fields=admin1"
        response = requests.get(url)
        data = response.json()
        events = data.get('data', [])
        region_counts = Counter([event['admin1'] for event in events])
    except Exception as e:
        print(f"Помилка при отриманні даних ACLED: {e}")
        region_counts = Counter()
    
    # Для аналізу загроз
    try:
        url_threats = f"https://api.acleddata.com/acled/read?key={api_key}&email={email}&country=Ukraine&event_type=Explosions/Remote violence&event_date={thirty_days_ago}&event_date_where=>&fields=sub_event_type"
        response_threats = requests.get(url_threats)
        data_threats = response_threats.json()
        threat_events = data_threats.get('data', [])
        sub_event_counts = Counter([event['sub_event_type'] for event in threat_events])
    except Exception as e:
        print(f"Помилка при отриманні даних загроз ACLED: {e}")
        sub_event_counts = Counter()
    
    return region_counts, sub_event_counts

region_counts, sub_event_counts = fetch_acled_data()

# Оновлення даних візуалізації
max_count = max(region_counts.values()) if region_counts else 1
for i, row in viz_data.iterrows():
    oblast = row['Область']
    english_name = region_name_mapping.get(oblast)
    count = region_counts.get(english_name, 0)
    security_level = 100 - (count / max_count * 90) if max_count > 0 else 100
    viz_data.at[i, 'Рівень безпеки'] = round(security_level, 1)

# Оновлення даних загроз
threat_mapping = {
    'Shelling/artillery/missile attack': ['Ракетні удари', 'Артилерійські обстріли'],
    'Air/drone strike': ['Безпілотники']
}
threat_risk = {threat: 0 for threat in threats_data['Тип загрози']}
for sub_event, count in sub_event_counts.items():
    if sub_event in threat_mapping:
        for threat in threat_mapping[sub_event]:
            threat_risk[threat] = count
max_threat_count = max(threat_risk.values()) if any(threat_risk.values()) else 0
for i, row in threats_data.iterrows():
    threat = row['Тип загрози']
    if threat in threat_risk:
        if max_threat_count > 0:
            scaled_risk = (threat_risk[threat] / max_threat_count) * 10
        else:
            scaled_risk = 0
        threats_data.at[i, 'Рівень ризику'] = round(scaled_risk, 1)

# Створення графіків
def create_ukraine_map():
    # Перевіряємо, чи маємо правильний GeoJSON
    if 'features' in ukraine_geojson and len(ukraine_geojson['features']) > 0:
        # Використовуємо choropleth з GeoJSON
        try:
            fig = px.choropleth(
                viz_data,
                geojson=ukraine_geojson,
                locations='Region English',
                featureidkey='properties.shapeName',
                color='Рівень безпеки',
                color_continuous_scale=[
                    [0, 'rgb(220, 53, 69)'],   # Червоний для низьких значень
                    [0.4, 'rgb(255, 193, 7)'], # Жовтий для середніх значень
                    [1, 'rgb(40, 167, 69)']    # Зелений для високих значень
                ],
                range_color=(10, 100),
                labels={'Рівень безпеки': 'Рівень безпеки'},
                hover_name='Область',
                hover_data={'Region English': False, 'Рівень безпеки': True}
            )
            fig.update_geos(
                fitbounds="locations", 
                visible=False,
                showcoastlines=True, 
                coastlinecolor="RebeccaPurple",
                showland=True, 
                landcolor="LightGray"
            )
            fig.update_layout(
                margin={"r":0,"t":0,"l":0,"b":0},
                coloraxis_colorbar={
                    'title': 'Рівень безпеки',
                    'tickvals': [10, 40, 70, 100],
                    'ticktext': ['Критичний', 'Небезпечний', 'Середній', 'Безпечний'],
                    'lenmode': 'fraction',
                    'len': 0.75
                }
            )
            print("Карта створена з використанням GeoJSON")
            return fig
        except Exception as e:
            print(f"Помилка при створенні карти з GeoJSON: {e}")
            # Якщо виникла помилка, використовуємо запасний варіант
    
    # Запасний варіант - використовуємо scatter map з покращеним дизайном
    print("Використовуємо запасний варіант карти")
    fig = go.Figure()
    
    # Додаємо контур для представлення України
    ukraine_lons = [22, 40, 40, 22, 22]  # Координати longitude
    ukraine_lats = [44, 44, 52, 52, 44]  # Координати latitude
    
    fig.add_trace(go.Scatter(
        x=ukraine_lons,
        y=ukraine_lats,
        mode='lines',
        line=dict(width=2, color='rgba(70, 130, 180, 0.8)'),
        name='Контур України',
        showlegend=False
    ))
    
    # Додаємо точки для кожної області
    # Приблизні координати для областей (спрощено)
    region_coords = {
        'Київська': (30.5, 50.4),
        'Львівська': (24.0, 49.8),
        'Харківська': (36.2, 49.9),
        'Одеська': (30.7, 46.5),
        'Дніпропетровська': (35.0, 48.5),
        'Донецька': (37.8, 48.0),
        'Луганська': (39.3, 48.9),
        'Запорізька': (35.4, 47.8),
        'Херсонська': (33.5, 46.6),
        'Миколаївська': (32.0, 47.0),
        'Вінницька': (28.5, 49.2),
        'Полтавська': (34.6, 49.6),
        'Сумська': (34.8, 51.0),
        'Чернігівська': (31.3, 51.5),
        'Черкаська': (32.0, 49.4),
        'Хмельницька': (27.0, 49.4),
        'Рівненська': (26.2, 50.6),
        'Волинська': (25.0, 51.0),
        'Івано-Франківська': (24.7, 48.9),
        'Тернопільська': (25.6, 49.6),
        'Закарпатська': (22.3, 48.6),
        'Чернівецька': (25.9, 48.3),
        'Житомирська': (28.7, 50.3),
        'Кіровоградська': (32.3, 48.5)
    }
    
    # Додаємо маркери для областей з кольором залежно від рівня безпеки
    for i, row in viz_data.iterrows():
        region = row['Область']
        security = row['Рівень безпеки']
        
        if region in region_coords:
            lon, lat = region_coords[region]
            
            # Визначаємо колір (червоний для небезпечних, жовтий для середніх, зелений для безпечних)
            if security < 40:
                marker_color = 'rgb(220, 53, 69)'  # Червоний
                security_text = 'Критичний'
            elif security < 70:
                marker_color = 'rgb(255, 193, 7)'  # Жовтий
                security_text = 'Середній'
            else:
                marker_color = 'rgb(40, 167, 69)'  # Зелений
                security_text = 'Безпечний'
                
            fig.add_trace(go.Scatter(
                x=[lon],
                y=[lat],
                mode='markers+text',
                marker=dict(
                    size=20, 
                    color=marker_color,
                    line=dict(width=2, color='white')
                ),
                text=region,
                textposition="top center",
                hoverinfo='text',
                hovertext=f"{region}<br>Рівень безпеки: {security} ({security_text})",
                name=f"{region} ({security})",
                showlegend=False
            ))
    
    fig.update_layout(
        margin={"r":0,"t":0,"l":0,"b":0},
        plot_bgcolor='rgba(240, 240, 240, 0.5)',
        paper_bgcolor='rgba(0,0,0,0)',
        geo=dict(
            projection_type='mercator',
            showland=True,
            landcolor='rgb(240, 240, 240)',
            countrycolor='rgb(204, 204, 204)'
        ),
        hoverlabel=dict(
            bgcolor="white",
            font_size=12,
            font_family="Arial"
        )
    )
    return fig

def create_threat_analysis():
    # Сортуємо дані за рівнем ризику для кращої візуалізації
    sorted_threats = threats_data.sort_values('Рівень ризику', ascending=False)
    
    fig = px.bar(
        sorted_threats,
        x='Тип загрози',
        y='Рівень ризику',
        color='Категорія',
        color_discrete_map={
            'Військова загроза': '#dc3545',      # Червоний
            'Кібер загроза': '#0dcaf0',         # Блакитний
            'Фізична загроза': '#fd7e14',       # Оранжевий
            'Інформаційна загроза': '#6f42c1',  # Фіолетовий
            'Критична інфраструктура': '#ffc107' # Жовтий
        },
        title='',
        labels={
            'Тип загрози': 'Тип загрози',
            'Рівень ризику': 'Рівень загрози (1-10)',
            'Категорія': 'Категорія загрози'
        },
        text='Рівень ризику'
    )
    
    # Покращення дизайну графіка
    fig.update_traces(
        texttemplate='%{text:.1f}',
        textposition='outside',
        marker_line_color='rgb(255, 255, 255)',
        marker_line_width=1.5,
        opacity=0.9
    )
    
    fig.update_layout(
        margin=dict(l=10, r=10, t=10, b=10),
        xaxis=dict(
            tickangle=45,
            title_font=dict(size=14),
            tickfont=dict(size=12),
            gridcolor='rgba(230, 230, 230, 0.5)'
        ),
        yaxis=dict(
            title_font=dict(size=14),
            tickfont=dict(size=12),
            gridcolor='rgba(230, 230, 230, 0.5)',
            range=[0, 10.5]  # Встановлюємо діапазон для осі Y
        ),
        plot_bgcolor='rgba(255, 255, 255, 0.9)',
        paper_bgcolor='rgba(0, 0, 0, 0)',
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1,
            font=dict(size=12)
        ),
        hoverlabel=dict(
            bgcolor="white",
            font_size=12,
            font_family="Arial"
        )
    )
    
    # Додаємо горизонтальні лінії для позначення рівнів ризику
    fig.add_shape(
        type="line",
        x0=-0.5,
        y0=7,
        x1=len(sorted_threats)-0.5,
        y1=7,
        line=dict(color="rgba(220, 53, 69, 0.5)", width=2, dash="dash"),
    )
    
    fig.add_shape(
        type="line",
        x0=-0.5,
        y0=4,
        x1=len(sorted_threats)-0.5,
        y1=4,
        line=dict(color="rgba(255, 193, 7, 0.5)", width=2, dash="dash"),
    )
    
    # Додаємо анотації для рівнів ризику
    fig.add_annotation(
        x=len(sorted_threats)-0.5,
        y=7,
        text="Високий ризик",
        showarrow=False,
        font=dict(size=10, color="rgba(220, 53, 69, 0.8)"),
        xshift=50,
        yshift=0
    )
    
    fig.add_annotation(
        x=len(sorted_threats)-0.5,
        y=4,
        text="Середній ризик",
        showarrow=False,
        font=dict(size=10, color="rgba(255, 193, 7, 0.8)"),
        xshift=50,
        yshift=0
    )
    
    return fig

def create_resource_management():
    # Сортуємо ресурси за рівнем використання
    sorted_resources = resources_data.sort_values('Використання (%)', ascending=False)
    
    # Створюємо покращений графік з індикаторами
    fig = go.Figure()
    
    # Визначаємо кількість рядків і стовпців для сітки
    num_resources = len(sorted_resources)
    cols = 3
    rows = (num_resources + cols - 1) // cols  # Округлення вгору
    
    # Розраховуємо розміри для кожного індикатора
    for i, row in sorted_resources.iterrows():
        resource = row['Ресурс']
        usage = row['Використання (%)']
        
        # Визначаємо колір індикатора залежно від рівня використання
        if usage > 80:
            color = "#dc3545"  # Червоний (критичний)
        elif usage > 60:
            color = "#fd7e14"  # Оранжевий (високий)
        else:
            color = "#28a745"  # Зелений (нормальний)
        
        # Розраховуємо позицію для індикатора в сітці
        row_idx = i // cols
        col_idx = i % cols
        
        # Розраховуємо домен для індикатора
        x_domain = [col_idx/cols, (col_idx+0.9)/cols]
        y_domain = [1 - (row_idx+1)/rows, 1 - row_idx/rows]
        
        fig.add_trace(go.Indicator(
            mode="gauge+number+delta",
            value=usage,
            domain={'x': x_domain, 'y': y_domain},
            title={
                'text': resource,
                'font': {'size': 14, 'color': colors['text']}
            },
            delta={'reference': 50, 'increasing': {'color': "#dc3545"}, 'decreasing': {'color': "#28a745"}},
            gauge={
                'axis': {'range': [None, 100], 'tickwidth': 1, 'tickcolor': colors['text']},
                'bar': {'color': color},
                'bgcolor': "white",
                'borderwidth': 2,
                'bordercolor': "gray",
                'steps': [
                    {'range': [0, 60], 'color': 'rgba(40, 167, 69, 0.2)'},
                    {'range': [60, 80], 'color': 'rgba(253, 126, 20, 0.3)'},
                    {'range': [80, 100], 'color': 'rgba(220, 53, 69, 0.3)'}
                ],
                'threshold': {
                    'line': {'color': "red", 'width': 4},
                    'thickness': 0.75,
                    'value': 90
                }
            }
        ))
    
    # Оновлюємо макет
    fig.update_layout(
        grid={'rows': rows, 'columns': cols, 'pattern': "independent"},
        margin=dict(l=20, r=20, t=30, b=20),
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
        height=rows*150,  # Динамічна висота залежно від кількості рядків
        showlegend=False
    )
    
    return fig

def create_forecast_chart():
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=forecast_data['Дата'],
        y=forecast_data['Фактично'],
        mode='markers+lines',
        name='Фактичні дані',
        line=dict(color='blue')
    ))
    fig.add_trace(go.Scatter(
        x=forecast_data['Дата'],
        y=forecast_data['Прогноз'],
        mode='lines',
        name='Прогноз',
        line=dict(color='red', dash='dash')
    ))
    fig.update_layout(
        title='Прогнозування потреб у ресурсах',
        xaxis_title='Дата',
        yaxis_title='Значення',
        margin=dict(l=0, r=0, t=30, b=0)
    )
    return fig

# Макет додатку
app.layout = html.Div(style={'backgroundColor': colors['background'], 'padding': '20px'}, children=[
    html.H1(
        'Компоненти панелі управління',
        style={
            'textAlign': 'center',
            'color': colors['text'],
            'fontFamily': 'Arial',
            'marginBottom': '30px'
        }
    ),
    
    html.Div(style={'display': 'flex', 'flexWrap': 'wrap', 'justifyContent': 'space-around'}, children=[
        html.Div(
            style={
                'backgroundColor': colors['visualization'],
                'padding': '15px',
                'borderRadius': '15px 15px 50px 50px',
                'width': '22%',
                'boxShadow': '0px 4px 8px rgba(0, 0, 0, 0.1)',
                'marginBottom': '20px'
            },
            children=[
                html.H3('Візуалізація', style={'textAlign': 'center', 'color': '#B58500'}),
                html.P('Відображає оперативну ситуацію в реальному часі.', style={'textAlign': 'center'}),
                dcc.Graph(
                    id='ukraine-map',
                    figure=create_ukraine_map(),
                    style={'height': '200px'}
                ),
                html.Div(id='live-update-text'),
                dcc.Interval(
                    id='interval-component',
                    interval=60*1000,  # оновлення кожну хвилину
                    n_intervals=0
                )
            ]
        ),
        
        html.Div(
            style={
                'backgroundColor': colors['threats'],
                'padding': '15px',
                'borderRadius': '15px 15px 50px 50px',
                'width': '22%',
                'boxShadow': '0px 4px 8px rgba(0, 0, 0, 0.1)',
                'marginBottom': '20px'
            },
            children=[
                html.H3('Аналіз загроз', style={'textAlign': 'center', 'color': '#B54300'}),
                html.P('Оцінює потенційні ризики та загрози.', style={'textAlign': 'center'}),
                dcc.Graph(
                    id='threat-analysis',
                    figure=create_threat_analysis(),
                    style={'height': '200px'}
                )
            ]
        ),
        
        html.Div(
            style={
                'backgroundColor': colors['resources'],
                'padding': '15px',
                'borderRadius': '15px 15px 50px 50px',
                'width': '22%',
                'boxShadow': '0px 4px 8px rgba(0, 0, 0, 0.1)',
                'marginBottom': '20px'
            },
            children=[
                html.H3('Управління ресурсами', style={'textAlign': 'center', 'color': '#B50000'}),
                html.P('Контролює розподіл та використання ресурсів.', style={'textAlign': 'center'}),
                dcc.Graph(
                    id='resource-management',
                    figure=create_resource_management(),
                    style={'height': '200px'}
                ),
                html.Div(
                    id='resource-alerts',
                    style={'marginTop': '10px', 'color': 'red', 'textAlign': 'center'}
                )
            ]
        ),
        
        html.Div(
            style={
                'backgroundColor': colors['forecasting'],
                'padding': '15px',
                'borderRadius': '15px 15px 50px 50px',
                'width': '22%',
                'boxShadow': '0px 4px 8px rgba(0, 0, 0, 0.1)',
                'marginBottom': '20px'
            },
            children=[
                html.H3('Інструменти прогнозування', style={'textAlign': 'center', 'color': '#B5006A'}),
                html.P('Надає інструменти для прогнозного аналізу.', style={'textAlign': 'center'}),
                dcc.Graph(
                    id='forecast-chart',
                    figure=create_forecast_chart(),
                    style={'height': '200px'}
                )
            ]
        )
    ]),
    
    html.Div(style={'marginTop': '30px'}, children=[
        html.H2('Деталізована інформація', style={'textAlign': 'center', 'color': colors['text']}),
        dcc.Tabs([
            dcc.Tab(label='Карта України', children=[
                html.Div(style={'padding': '20px'}, children=[
                    dcc.Graph(figure=create_ukraine_map())
                ])
            ]),
            dcc.Tab(label='Аналіз загроз війни', children=[
                html.Div(style={'padding': '20px'}, children=[
                    dcc.Graph(figure=create_threat_analysis()),
                    html.Div([
                        html.H4('Контекст Російсько-Української війни'),
                        html.P('Дашборд відображає актуальні дані по різним типам загроз, пов\'язаних з воєнними діями. '
                               'Моніторинг включає як військові загрози (обстріли, атаки), так і супутні ризики для '
                               'цивільної інфраструктури та інформаційної безпеки.')
                    ])
                ])
            ]),
            dcc.Tab(label='Управління ресурсами', children=[
                html.Div(style={'padding': '20px'}, children=[
                    dcc.Graph(figure=create_resource_management())
                ])
            ]),
            dcc.Tab(label='Прогнозування', children=[
                html.Div(style={'padding': '20px'}, children=[
                    dcc.Graph(figure=create_forecast_chart())
                ])
            ])
        ])
    ])
])

# Оновлення інформації в реальному часі
@app.callback(
    Output('live-update-text', 'children'),
    Input('interval-component', 'n_intervals')
)
def update_live_info(n):
    return html.Div([
        html.P(f"Останнє оновлення: {pd.Timestamp.now().strftime('%H:%M:%S')}")
    ])

# Оновлення повідомлень про стан ресурсів
@app.callback(
    Output('resource-alerts', 'children'),
    Input('interval-component', 'n_intervals')
)
def update_resource_alerts(n):
    critical_resources = resources_data[resources_data['Використання (%)'] > 80]['Ресурс'].tolist()
    if critical_resources:
        return html.P(f"Критичний рівень: {', '.join(critical_resources)}")
    return html.P("Усі ресурси в нормі")

# Запуск сервера
print("Запуск сервера Dash...")
print("Відкрийте http://127.0.0.1:8050/ у вашому браузері для перегляду дашборду.")


Помилка при завантаженні GeoJSON: Extra data: line 1 column 4 (char 3)
Створено запасний GeoJSON
Карта створена з використанням GeoJSON


C:\Users\TZ\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



Карта створена з використанням GeoJSON
Запуск сервера Dash...
Відкрийте http://127.0.0.1:8050/ у вашому браузері для перегляду дашборду.


C:\Users\TZ\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [ ]:
# Вдосконалений дашборд для інформаційно-аналітичної системи Національної гвардії України
# Інтеграція з API-ендпоінтами та покращена візуалізація

import dash
from dash import dcc, html
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from dash.dependencies import Input, Output, State
import requests
from datetime import datetime, timedelta
from collections import Counter
import json
import os

# Ініціалізація додатку Dash з покращеними метаданими
app = dash.Dash(
    __name__,
    meta_tags=[
        {"name": "viewport", "content": "width=device-width, initial-scale=1.0"}
    ],
    title="Інформаційно-аналітична система НГУ",
    suppress_callback_exceptions=True,
    external_stylesheets=[
        "https://cdn.jsdelivr.net/npm/bootstrap@5.1.3/dist/css/bootstrap.min.css",
        "https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.0.0-beta3/css/all.min.css",
        "https://cdnjs.cloudflare.com/ajax/libs/animate.css/4.1.1/animate.min.css"
    ]
)

# Покращена кольорова схема з національними кольорами України
colors = {
    'visualization': '#E3F2FD',  # світло-блакитний
    'threats': '#FFF3E0',        # світло-оранжевий
    'resources': '#E8F5E9',      # світло-зелений
    'forecasting': '#F3E5F5',    # світло-фіолетовий
    'background': '#FAFAFA',     # світло-сірий фон
    'text': '#37474F',           # темно-сірий текст
    'accent': '#0057b7',         # синій (національний колір України)
    'accent2': '#ffd700',        # жовтий (національний колір України)
    'warning': '#FF9800',        # оранжевий для попереджень
    'danger': '#F44336',         # червоний для небезпеки
    'success': '#4CAF50'         # зелений для успіху
}

# Базовий URL для API-ендпоінтів
API_BASE_URL = "http://localhost:5000/api"

# Функції для отримання даних з API
def get_dashboard_statistics():
    try:
        response = requests.get(f"{API_BASE_URL}/dashboard/statistics")
        return response.json()
    except Exception as e:
        print(f"Помилка при отриманні статистики: {e}")
        # Повертаємо тестові дані у випадку помилки
        return {
            'units_count': 25,
            'resources_count': 150,
            'incidents_count': 42,
            'critical_situations_count': 3
        }

def get_recent_incidents():
    try:
        response = requests.get(f"{API_BASE_URL}/incidents/recent")
        return response.json()
    except Exception as e:
        print(f"Помилка при отриманні інцидентів: {e}")
        # Повертаємо тестові дані у випадку помилки
        return [
            {
                'id': 1,
                'type': 'Порушення громадського порядку',
                'subtype': 'Масові заворушення',
                'location': 'Київ',
                'timestamp': datetime.now().isoformat(),
                'severity': 'Високий',
                'status': 'Активний'
            }
        ]

def get_threat_level_history():
    try:
        response = requests.get(f"{API_BASE_URL}/analytics/threat-level-history")
        return response.json()
    except Exception as e:
        print(f"Помилка при отриманні історії рівня загрози: {e}")
        # Повертаємо тестові дані у випадку помилки
        dates = [(datetime.now() - timedelta(days=i)).strftime('%d.%m.%Y') for i in range(30, 0, -1)]
        return {
            'dates': dates,
            'threat_levels': [np.random.randint(30, 80) for _ in range(30)]
        }

def get_resources_distribution():
    try:
        response = requests.get(f"{API_BASE_URL}/resources/distribution")
        return response.json()
    except Exception as e:
        print(f"Помилка при отриманні розподілу ресурсів: {e}")
        # Повертаємо тестові дані у випадку помилки
        resources = [
            'Військове обладнання', 'Гуманітарна допомога', 'Енергетичні ресурси',
            'Медичні засоби', 'Системи зв\'язку', 'Транспорт', 'Продовольство'
        ]
        return {
            'resources': [
                {'id': i+1, 'name': res, 'type': res.split()[0], 'usage_percentage': np.random.randint(50, 95)}
                for i, res in enumerate(resources)
            ],
            'by_type': {
                res.split()[0]: {'count': np.random.randint(10, 50), 'percentage': np.random.randint(10, 30)}
                for res in resources
            },
            'by_status': {
                'Доступний': {'count': 120, 'percentage': 80},
                'В ремонті': {'count': 15, 'percentage': 10},
                'Недоступний': {'count': 15, 'percentage': 10}
            }
        }

def get_map_data():
    try:
        response = requests.get(f"{API_BASE_URL}/situations/map-data")
        return response.json()
    except Exception as e:
        print(f"Помилка при отриманні даних для карти: {e}")
        # Повертаємо тестові дані у випадку помилки
        regions = [
            'Київська', 'Львівська', 'Харківська', 'Одеська', 'Дніпропетровська',
            'Донецька', 'Луганська', 'Запорізька', 'Херсонська'
        ]
        statuses = ['Нормальна', 'Напружена', 'Критична']
        return [
            {
                'id': i+1,
                'location': region,
                'region': region,
                'status': np.random.choice(statuses),
                'threat_level': np.random.randint(1, 10),
                'timestamp': datetime.now().isoformat(),
                'coordinates': f"{np.random.uniform(44.0, 52.0):.6f}, {np.random.uniform(22.0, 40.0):.6f}"
            }
            for i, region in enumerate(regions)
        ]

def get_ai_predictions():
    try:
        response = requests.get(f"{API_BASE_URL}/analytics/predictions")
        return response.json()
    except Exception as e:
        print(f"Помилка при отриманні прогнозів AI: {e}")
        # Повертаємо тестові дані у випадку помилки
        return [
            {
                'id': 1,
                'type': 'Загроза безпеці',
                'region': 'Харківська',
                'location': 'Харків',
                'value': 8.5,
                'confidence': 0.85,
                'description': 'Прогнозується підвищення рівня загрози в регіоні',
                'timestamp': datetime.now().isoformat()
            }
        ]

# Функції для створення графіків
def create_ukraine_map(map_data=None):
    if map_data is None:
        map_data = get_map_data()
    
    # Створюємо базову карту
    fig = go.Figure()
    
    # Додаємо маркери для кожної ситуації
    for situation in map_data:
        # Парсимо координати
        try:
            lat, lon = map(float, situation['coordinates'].split(','))
        except:
            continue
        
        # Визначаємо колір маркера залежно від статусу
        if situation['status'] == 'Критична':
            marker_color = colors['danger']
        elif situation['status'] == 'Напружена':
            marker_color = colors['warning']
        else:
            marker_color = colors['success']
        
        # Додаємо маркер
        fig.add_trace(go.Scattergeo(
            lon=[lon],
            lat=[lat],
            text=situation['location'],
            mode='markers',
            marker=dict(
                size=10,
                color=marker_color,
                line=dict(width=1, color='white')
            ),
            hoverinfo='text',
            hovertext=f"<b>{situation['location']}</b><br>" +
                      f"Статус: {situation['status']}<br>" +
                      f"Рівень загрози: {situation['threat_level']}/10<br>" +
                      f"Оновлено: {situation['timestamp'].split('T')[0]}"
        ))
    
    # Налаштовуємо вигляд карти
    fig.update_geos(
        visible=False,
        resolution=50,
        scope="europe",
        showcountries=True,
        countrycolor="gray",
        showsubunits=True,
        subunitcolor="blue",
        center=dict(lon=31.1656, lat=48.3794),  # Центр України
        projection_scale=5
    )
    
    fig.update_layout(
        title={
            'text': 'Карта оперативної обстановки',
            'y': 0.95,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': {'size': 20, 'color': colors['text']}
        },
        margin={"r":0, "t":30, "l":0, "b":0},
        height=500,
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        geo=dict(
            bgcolor='rgba(0,0,0,0)'
        )
    )
    
    return fig

def create_threat_analysis():
    # Отримуємо дані про рівень загрози
    threat_data = get_threat_level_history()
    
    # Створюємо графік
    fig = go.Figure()
    
    # Додаємо лінію рівня загрози
    fig.add_trace(go.Scatter(
        x=threat_data['dates'],
        y=threat_data['threat_levels'],
        mode='lines+markers',
        name='Рівень загрози',
        line=dict(color=colors['accent'], width=3),
        marker=dict(size=8, color=colors['accent']),
        fill='tozeroy',
        fillcolor=f"rgba({int(colors['accent'][1:3], 16)}, {int(colors['accent'][3:5], 16)}, {int(colors['accent'][5:7], 16)}, 0.2)"
    ))
    
    # Додаємо зони рівнів загрози
    fig.add_shape(
        type="rect",
        x0=0,
        y0=70,
        x1=len(threat_data['dates']),
        y1=100,
        fillcolor="rgba(76, 175, 80, 0.1)",
        line=dict(width=0),
        layer="below"
    )
    
    fig.add_shape(
        type="rect",
        x0=0,
        y0=40,
        x1=len(threat_data['dates']),
        y1=70,
        fillcolor="rgba(255, 152, 0, 0.1)",
        line=dict(width=0),
        layer="below"
    )
    
    fig.add_shape(
        type="rect",
        x0=0,
        y0=0,
        x1=len(threat_data['dates']),
        y1=40,
        fillcolor="rgba(244, 67, 54, 0.1)",
        line=dict(width=0),
        layer="below"
    )
    
    # Додаємо анотації для зон
    fig.add_annotation(
        x=len(threat_data['dates']) - 1,
        y=85,
        text="Низький рівень загрози",
        showarrow=False,
        font=dict(size=10, color="rgba(76, 175, 80, 0.7)")
    )
    
    fig.add_annotation(
        x=len(threat_data['dates']) - 1,
        y=55,
        text="Середній рівень загрози",
        showarrow=False,
        font=dict(size=10, color="rgba(255, 152, 0, 0.7)")
    )
    
    fig.add_annotation(
        x=len(threat_data['dates']) - 1,
        y=20,
        text="Високий рівень загрози",
        showarrow=False,
        font=dict(size=10, color="rgba(244, 67, 54, 0.7)")
    )
    
    # Налаштовуємо вигляд графіка
    fig.update_layout(
        title={
            'text': 'Динаміка рівня загрози',
            'y': 0.95,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': {'size': 20, 'color': colors['text']}
        },
        xaxis=dict(
            title='Дата',
            tickangle=45,
            tickmode='array',
            tickvals=[i for i in range(0, len(threat_data['dates']), 5)],
            ticktext=[threat_data['dates'][i] for i in range(0, len(threat_data['dates']), 5)],
            gridcolor='rgba(230, 230, 230, 0.5)'
        ),
        yaxis=dict(
            title='Рівень загрози',
            range=[0, 100],
            gridcolor='rgba(230, 230, 230, 0.5)'
        ),
        margin=dict(l=40, r=40, t=50, b=40),
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        hovermode='x unified'
    )
    
    return fig

def create_resource_management():
    # Отримуємо дані про ресурси
    resource_data = get_resources_distribution()
    
    # Створюємо дані для графіка
    resource_types = list(resource_data['by_type'].keys())
    resource_counts = [data['count'] for data in resource_data['by_type'].values()]
    resource_percentages = [data['percentage'] for data in resource_data['by_type'].values()]
    
    # Створюємо графік розподілу ресурсів за типами
    fig = go.Figure()
    
    # Додаємо стовпчики для кожного типу ресурсів
    fig.add_trace(go.Bar(
        x=resource_types,
        y=resource_counts,
        marker_color=colors['accent'],
        text=resource_percentages,
        texttemplate='%{text}%',
        textposition='outside',
        hovertemplate='<b>%{x}</b><br>Кількість: %{y}<br>Відсоток: %{text}%<extra></extra>'
    ))
    
    # Налаштовуємо вигляд графіка
    fig.update_layout(
        title={
            'text': 'Розподіл ресурсів за типами',
            'y': 0.95,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': {'size': 20, 'color': colors['text']}
        },
        xaxis=dict(
            title='Тип ресурсу',
            tickangle=45,
            gridcolor='rgba(230, 230, 230, 0.5)'
        ),
        yaxis=dict(
            title='Кількість',
            gridcolor='rgba(230, 230, 230, 0.5)'
        ),
        margin=dict(l=40, r=40, t=50, b=40),
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)'
    )
    
    return fig

def create_resource_status_chart():
    # Отримуємо дані про ресурси
    resource_data = get_resources_distribution()
    
    # Створюємо дані для графіка
    status_labels = list(resource_data['by_status'].keys())
    status_values = [data['percentage'] for data in resource_data['by_status'].values()]
    
    # Визначаємо кольори для статусів
    status_colors = {
        'Доступний': colors['success'],
        'В ремонті': colors['warning'],
        'Недоступний': colors['danger']
    }
    
    # Створюємо графік розподілу ресурсів за статусами
    fig = go.Figure()
    
    # Додаємо кругову діаграму
    fig.add_trace(go.Pie(
        labels=status_labels,
        values=status_values,
        marker=dict(
            colors=[status_colors.get(status, colors['accent']) for status in status_labels],
            line=dict(color='white', width=2)
        ),
        textinfo='label+percent',
        insidetextorientation='radial',
        hole=0.4
    ))
    
    # Налаштовуємо вигляд графіка
    fig.update_layout(
        title={
            'text': 'Статус ресурсів',
            'y': 0.95,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': {'size': 20, 'color': colors['text']}
        },
        margin=dict(l=20, r=20, t=50, b=20),
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=-0.1,
            xanchor="center",
            x=0.5
        )
    )
    
    return fig

def create_incidents_chart():
    # Отримуємо дані про інциденти
    incidents = get_recent_incidents()
    
    # Групуємо інциденти за типами
    incident_types = {}
    for incident in incidents:
        incident_type = incident['type']
        if incident_type in incident_types:
            incident_types[incident_type] += 1
        else:
            incident_types[incident_type] = 1
    
    # Створюємо дані для графіка
    types = list(incident_types.keys())
    counts = list(incident_types.values())
    
    # Створюємо графік інцидентів
    fig = go.Figure()
    
    # Додаємо стовпчики для кожного типу інцидентів
    fig.add_trace(go.Bar(
        x=types,
        y=counts,
        marker_color=colors['warning'],
        text=counts,
        textposition='outside',
        hovertemplate='<b>%{x}</b><br>Кількість: %{y}<extra></extra>'
    ))
    
    # Налаштовуємо вигляд графіка
    fig.update_layout(
        title={
            'text': 'Розподіл інцидентів за типами',
            'y': 0.95,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': {'size': 20, 'color': colors['text']}
        },
        xaxis=dict(
            title='Тип інциденту',
            tickangle=45,
            gridcolor='rgba(230, 230, 230, 0.5)'
        ),
        yaxis=dict(
            title='Кількість',
            gridcolor='rgba(230, 230, 230, 0.5)'
        ),
        margin=dict(l=40, r=40, t=50, b=40),
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)'
    )
    
    return fig

# Макет додатку
app.layout = html.Div(style={'backgroundColor': colors['background'], 'minHeight': '100vh'}, children=[
    # Заголовок
    html.Div(className="container-fluid", children=[
        html.Div(className="row mb-4", children=[
            html.Div(className="col-12", children=[
                html.H1("ІНФОРМАЦІЙНО-АНАЛІТИЧНА СИСТЕМА НГУ", className="display-5 fw-bold text-center mt-4 animate__animated animate__fadeIn"),
                html.P("Інтерактивний дашборд для моніторингу та управління ресурсами Національної гвардії України", 
                       className="lead text-muted text-center animate__animated animate__fadeIn")
            ])
        ])
    ]),
    
    # Статистика
    html.Div(className="container-fluid", children=[
        html.Div(className="row g-4 mb-4", children=[
            # Кількість підрозділів
            html.Div(className="col-md-6 col-lg-3", children=[
                html.Div(className="card h-100 animate__animated animate__fadeInUp", children=[
                    html.Div(className="card-body text-center", children=[
                        html.Div(className="d-flex align-items-center justify-content-center mb-3", children=[
                            html.I(className="fas fa-users fa-3x text-primary me-3"),
                            html.H2(id="units-count", className="display-4 mb-0")
                        ]),
                        html.H3("Підрозділи", className="card-title h5 mb-2"),
                        html.P("Загальна кількість підрозділів НГУ", className="card-text text-muted")
                    ])
                ])
            ]),
            
            # Кількість ресурсів
            html.Div(className="col-md-6 col-lg-3", children=[
                html.Div(className="card h-100 animate__animated animate__fadeInUp", 
                         style={"animation-delay": "0.1s"}, children=[
                    html.Div(className="card-body text-center", children=[
                        html.Div(className="d-flex align-items-center justify-content-center mb-3", children=[
                            html.I(className="fas fa-boxes fa-3x text-success me-3"),
                            html.H2(id="resources-count", className="display-4 mb-0")
                        ]),
                        html.H3("Ресурси", className="card-title h5 mb-2"),
                        html.P("Доступні ресурси в системі", className="card-text text-muted")
                    ])
                ])
            ]),
            
            # Кількість інцидентів
            html.Div(className="col-md-6 col-lg-3", children=[
                html.Div(className="card h-100 animate__animated animate__fadeInUp", 
                         style={"animation-delay": "0.2s"}, children=[
                    html.Div(className="card-body text-center", children=[
                        html.Div(className="d-flex align-items-center justify-content-center mb-3", children=[
                            html.I(className="fas fa-exclamation-triangle fa-3x text-warning me-3"),
                            html.H2(id="incidents-count", className="display-4 mb-0")
                        ]),
                        html.H3("Інциденти", className="card-title h5 mb-2"),
                        html.P("Зареєстровані інциденти за 30 днів", className="card-text text-muted")
                    ])
                ])
            ]),
            
            # Критичні ситуації
            html.Div(className="col-md-6 col-lg-3", children=[
                html.Div(className="card h-100 animate__animated animate__fadeInUp", 
                         style={"animation-delay": "0.3s"}, children=[
                    html.Div(className="card-body text-center", children=[
                        html.Div(className="d-flex align-items-center justify-content-center mb-3", children=[
                            html.I(className="fas fa-radiation fa-3x text-danger me-3"),
                            html.H2(id="critical-count", className="display-4 mb-0")
                        ]),
                        html.H3("Критичні ситуації", className="card-title h5 mb-2"),
                        html.P("Активні критичні ситуації", className="card-text text-muted")
                    ])
                ])
            ])
        ])
    ]),
    
    # Основні графіки
    html.Div(className="container-fluid", children=[
        html.Div(className="row g-4 mb-4", children=[
            # Карта України
            html.Div(className="col-lg-8", children=[
                html.Div(className="card h-100 animate__animated animate__fadeIn", 
                         style={"animation-delay": "0.4s"}, children=[
                    html.Div(className="card-header d-flex justify-content-between align-items-center", children=[
                        html.H5("Карта оперативної обстановки", className="mb-0"),
                        html.Button("Оновити", id="update-map", className="btn btn-sm btn-outline-primary")
                    ]),
                    html.Div(className="card-body p-0", children=[
                        dcc.Graph(id="ukraine-map", figure=create_ukraine_map(), config={'displayModeBar': False})
                    ])
                ])
            ]),
            
            # Аналіз загроз
            html.Div(className="col-lg-4", children=[
                html.Div(className="card h-100 animate__animated animate__fadeIn", 
                         style={"animation-delay": "0.5s"}, children=[
                    html.Div(className="card-header", children=[
                        html.H5("Аналіз загроз", className="mb-0")
                    ]),
                    html.Div(className="card-body", children=[
                        dcc.Graph(id="threat-analysis", figure=create_threat_analysis(), config={'displayModeBar': False})
                    ])
                ])
            ])
        ]),
        
        # Другий ряд графіків
        html.Div(className="row g-4 mb-4", children=[
            # Управління ресурсами
            html.Div(className="col-lg-6", children=[
                html.Div(className="card h-100 animate__animated animate__fadeIn", 
                         style={"animation-delay": "0.6s"}, children=[
                    html.Div(className="card-header d-flex justify-content-between align-items-center", children=[
                        html.H5("Управління ресурсами", className="mb-0"),
                        html.Div(className="btn-group", children=[
                            html.Button("За типами", id="resource-type-btn", className="btn btn-sm btn-outline-primary active"),
                            html.Button("За статусом", id="resource-status-btn", className="btn btn-sm btn-outline-primary")
                        ])
                    ]),
                    html.Div(className="card-body", children=[
                        html.Div(id="resource-chart-container", children=[
                            dcc.Graph(id="resource-chart", figure=create_resource_management(), config={'displayModeBar': False})
                        ])
                    ])
                ])
            ]),
            
            # Інциденти
            html.Div(className="col-lg-6", children=[
                html.Div(className="card h-100 animate__animated animate__fadeIn", 
                         style={"animation-delay": "0.7s"}, children=[
                    html.Div(className="card-header", children=[
                        html.H5("Останні інциденти", className="mb-0")
                    ]),
                    html.Div(className="card-body", children=[
                        html.Div(id="incidents-table-container", style={"maxHeight": "400px", "overflow": "auto"}, children=[
                            html.Table(className="table table-hover", children=[
                                html.Thead(className="table-light", children=[
                                    html.Tr(children=[
                                        html.Th("Тип"),
                                        html.Th("Локація"),
                                        html.Th("Дата"),
                                        html.Th("Статус"),
                                        html.Th("Рівень")
                                    ])
                                ]),
                                html.Tbody(id="incidents-table-body")
                            ])
                        ])
                    ])
                ])
            ])
        ]),
        
        # Третій ряд - AI прогнози
        html.Div(className="row g-4 mb-4", children=[
            html.Div(className="col-12", children=[
                html.Div(className="card animate__animated animate__fadeIn", 
                         style={"animation-delay": "0.8s"}, children=[
                    html.Div(className="card-header d-flex justify-content-between align-items-center", children=[
                        html.H5("Прогнози штучного інтелекту", className="mb-0"),
                        html.Span(className="badge bg-primary", children=["AI"])
                    ]),
                    html.Div(className="card-body", children=[
                        html.Div(className="row", id="ai-predictions-container")
                    ])
                ])
            ])
        ])
    ]),
    
    # Інтервал для автоматичного оновлення даних
    dcc.Interval(
        id='interval-component',
        interval=60*1000,  # оновлення кожну хвилину
        n_intervals=0
    )
])

# Колбеки для оновлення даних
@app.callback(
    [
        Output("units-count", "children"),
        Output("resources-count", "children"),
        Output("incidents-count", "children"),
        Output("critical-count", "children")
    ],
    [Input("interval-component", "n_intervals")]
)
def update_statistics(n):
    """Оновлення статистичних даних"""
    stats = get_dashboard_statistics()
    return [
        stats['units_count'],
        stats['resources_count'],
        stats['incidents_count'],
        stats['critical_situations_count']
    ]

@app.callback(
    Output("ukraine-map", "figure"),
    [Input("update-map", "n_clicks"), Input("interval-component", "n_intervals")]
)
def update_map(n_clicks, n_intervals):
    """Оновлення карти"""
    return create_ukraine_map()

@app.callback(
    Output("threat-analysis", "figure"),
    [Input("interval-component", "n_intervals")]
)
def update_threat_analysis(n):
    """Оновлення аналізу загроз"""
    return create_threat_analysis()

@app.callback(
    Output("resource-chart-container", "children"),
    [Input("resource-type-btn", "n_clicks"), Input("resource-status-btn", "n_clicks")]
)
def update_resource_chart(type_clicks, status_clicks):
    """Перемикання між різними видами графіків ресурсів"""
    ctx = dash.callback_context
    if not ctx.triggered:
        # За замовчуванням показуємо розподіл за типами
        return [dcc.Graph(id="resource-chart", figure=create_resource_management(), config={'displayModeBar': False})]
    else:
        button_id = ctx.triggered[0]['prop_id'].split('.')[0]
        if button_id == "resource-status-btn":
            return [dcc.Graph(id="resource-chart", figure=create_resource_status_chart(), config={'displayModeBar': False})]
        else:
            return [dcc.Graph(id="resource-chart", figure=create_resource_management(), config={'displayModeBar': False})]

@app.callback(
    Output("incidents-table-body", "children"),
    [Input("interval-component", "n_intervals")]
)
def update_incidents_table(n):
    """Оновлення таблиці інцидентів"""
    incidents = get_recent_incidents()
    rows = []
    for incident in incidents:
        # Визначаємо клас для рядка залежно від рівня важливості
        severity_class = ""
        if incident['severity'] == 'Високий':
            severity_class = "table-danger"
        elif incident['severity'] == 'Середній':
            severity_class = "table-warning"
        
        # Форматуємо дату
        try:
            date_obj = datetime.fromisoformat(incident['timestamp'])
            formatted_date = date_obj.strftime("%d.%m.%Y %H:%M")
        except:
            formatted_date = incident['timestamp']
        
        # Додаємо рядок до таблиці
        rows.append(html.Tr(className=severity_class, children=[
            html.Td(f"{incident['type']} - {incident['subtype']}"),
            html.Td(incident['location']),
            html.Td(formatted_date),
            html.Td(incident['status']),
            html.Td(incident['severity'])
        ]))
    
    return rows

@app.callback(
    Output("ai-predictions-container", "children"),
    [Input("interval-component", "n_intervals")]
)
def update_ai_predictions(n):
    """Оновлення прогнозів AI"""
    predictions = get_ai_predictions()
    prediction_cards = []
    
    for prediction in predictions:
        # Визначаємо колір картки залежно від значення прогнозу
        if prediction['value'] > 7:
            card_class = "border-danger"
            header_class = "bg-danger text-white"
        elif prediction['value'] > 4:
            card_class = "border-warning"
            header_class = "bg-warning"
        else:
            card_class = "border-success"
            header_class = "bg-success text-white"
        
        # Форматуємо дату
        try:
            date_obj = datetime.fromisoformat(prediction['timestamp'])
            formatted_date = date_obj.strftime("%d.%m.%Y")
        except:
            formatted_date = prediction['timestamp']
        
        # Створюємо картку прогнозу
        prediction_cards.append(html.Div(className="col-md-4 mb-3", children=[
            html.Div(className=f"card h-100 {card_class}", children=[
                html.Div(className=f"card-header {header_class}", children=[
                    html.H6(prediction['type'], className="mb-0")
                ]),
                html.Div(className="card-body", children=[
                    html.H5(prediction['region'], className="card-title"),
                    html.P(prediction['description'], className="card-text"),
                    html.Div(className="d-flex justify-content-between align-items-center", children=[
                        html.Span(f"Значення: {prediction['value']}"),
                        html.Span(f"Достовірність: {int(prediction['confidence']*100)}%")
                    ])
                ]),
                html.Div(className="card-footer text-muted", children=[
                    f"Оновлено: {formatted_date}"
                ])
            ])
        ]))
    
    return prediction_cards

# НЕ ЗАПУСКАЄМО app.run_server() тут, оскільки інтеграція здійснюється через DispatcherMiddleware у Flask
if __name__ == '__main__':
    app.run_server(debug=True, port=8050)
    print("Сервер запущено. Відкрийте http://127.0.0.1:8050/ у вашому браузері.")

In [ ]:
# Вдосконалений дашборд для інформаційно-аналітичної системи Національної гвардії України
# Інтеграція з API-ендпоінтами та покращена візуалізація

import dash
from dash import dcc, html
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from dash.dependencies import Input, Output, State
import requests
from datetime import datetime, timedelta
from collections import Counter
import json
import os

# Ініціалізація додатку Dash з покращеними метаданими
app = dash.Dash(
    __name__,
    meta_tags=[
        {"name": "viewport", "content": "width=device-width, initial-scale=1.0"}
    ],
    title="Інформаційно-аналітична система НГУ",
    suppress_callback_exceptions=True,
    external_stylesheets=[
        "https://cdn.jsdelivr.net/npm/bootstrap@5.1.3/dist/css/bootstrap.min.css",
        "https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.0.0-beta3/css/all.min.css",
        "https://cdnjs.cloudflare.com/ajax/libs/animate.css/4.1.1/animate.min.css"
    ]
)

# Покращена кольорова схема з національними кольорами України
colors = {
    'visualization': '#E3F2FD',  # світло-блакитний
    'threats': '#FFF3E0',        # світло-оранжевий
    'resources': '#E8F5E9',      # світло-зелений
    'forecasting': '#F3E5F5',    # світло-фіолетовий
    'background': '#FAFAFA',     # світло-сірий фон
    'text': '#37474F',           # темно-сірий текст
    'accent': '#0057b7',         # синій (національний колір України)
    'accent2': '#ffd700',        # жовтий (національний колір України)
    'warning': '#FF9800',        # оранжевий для попереджень
    'danger': '#F44336',         # червоний для небезпеки
    'success': '#4CAF50'         # зелений для успіху
}

# Базовий URL для API-ендпоінтів
API_BASE_URL = "http://localhost:5000/api"

# Функції для отримання даних з API
def get_dashboard_statistics():
    try:
        response = requests.get(f"{API_BASE_URL}/dashboard/statistics")
        return response.json()
    except Exception as e:
        print(f"Помилка при отриманні статистики: {e}")
        # Повертаємо тестові дані у випадку помилки
        return {
            'units_count': 25,
            'resources_count': 150,
            'incidents_count': 42,
            'critical_situations_count': 3
        }

def get_recent_incidents():
    try:
        response = requests.get(f"{API_BASE_URL}/incidents/recent")
        return response.json()
    except Exception as e:
        print(f"Помилка при отриманні інцидентів: {e}")
        # Повертаємо тестові дані у випадку помилки
        return [
            {
                'id': 1,
                'type': 'Порушення громадського порядку',
                'subtype': 'Масові заворушення',
                'location': 'Київ',
                'timestamp': datetime.now().isoformat(),
                'severity': 'Високий',
                'status': 'Активний'
            }
        ]

def get_threat_level_history():
    try:
        response = requests.get(f"{API_BASE_URL}/analytics/threat-level-history")
        return response.json()
    except Exception as e:
        print(f"Помилка при отриманні історії рівня загрози: {e}")
        # Повертаємо тестові дані у випадку помилки
        dates = [(datetime.now() - timedelta(days=i)).strftime('%d.%m.%Y') for i in range(30, 0, -1)]
        return {
            'dates': dates,
            'threat_levels': [np.random.randint(30, 80) for _ in range(30)]
        }

def get_resources_distribution():
    try:
        response = requests.get(f"{API_BASE_URL}/resources/distribution")
        return response.json()
    except Exception as e:
        print(f"Помилка при отриманні розподілу ресурсів: {e}")
        # Повертаємо тестові дані у випадку помилки
        resources = [
            'Військове обладнання', 'Гуманітарна допомога', 'Енергетичні ресурси',
            'Медичні засоби', 'Системи зв\'язку', 'Транспорт', 'Продовольство'
        ]
        return {
            'resources': [
                {'id': i+1, 'name': res, 'type': res.split()[0], 'usage_percentage': np.random.randint(50, 95)}
                for i, res in enumerate(resources)
            ],
            'by_type': {
                res.split()[0]: {'count': np.random.randint(10, 50), 'percentage': np.random.randint(10, 30)}
                for res in resources
            },
            'by_status': {
                'Доступний': {'count': 120, 'percentage': 80},
                'В ремонті': {'count': 15, 'percentage': 10},
                'Недоступний': {'count': 15, 'percentage': 10}
            }
        }

def get_map_data():
    try:
        response = requests.get(f"{API_BASE_URL}/situations/map-data")
        return response.json()
    except Exception as e:
        print(f"Помилка при отриманні даних для карти: {e}")
        # Повертаємо тестові дані у випадку помилки
        regions = [
            'Київська', 'Львівська', 'Харківська', 'Одеська', 'Дніпропетровська',
            'Донецька', 'Луганська', 'Запорізька', 'Херсонська'
        ]
        statuses = ['Нормальна', 'Напружена', 'Критична']
        return [
            {
                'id': i+1,
                'location': region,
                'region': region,
                'status': np.random.choice(statuses),
                'threat_level': np.random.randint(1, 10),
                'timestamp': datetime.now().isoformat(),
                'coordinates': f"{np.random.uniform(44.0, 52.0):.6f}, {np.random.uniform(22.0, 40.0):.6f}"
            }
            for i, region in enumerate(regions)
        ]

def get_ai_predictions():
    try:
        response = requests.get(f"{API_BASE_URL}/analytics/predictions")
        return response.json()
    except Exception as e:
        print(f"Помилка при отриманні прогнозів AI: {e}")
        # Повертаємо тестові дані у випадку помилки
        return [
            {
                'id': 1,
                'type': 'Загроза безпеці',
                'region': 'Харківська',
                'location': 'Харків',
                'value': 8.5,
                'confidence': 0.85,
                'description': 'Прогнозується підвищення рівня загрози в регіоні',
                'timestamp': datetime.now().isoformat()
            }
        ]

# Функції для створення графіків
def create_ukraine_map(map_data=None):
    if map_data is None:
        map_data = get_map_data()
    
    # Створюємо базову карту
    fig = go.Figure()
    
    # Додаємо маркери для кожної ситуації
    for situation in map_data:
        # Парсимо координати
        try:
            lat, lon = map(float, situation['coordinates'].split(','))
        except:
            continue
        
        # Визначаємо колір маркера залежно від статусу
        if situation['status'] == 'Критична':
            marker_color = colors['danger']
        elif situation['status'] == 'Напружена':
            marker_color = colors['warning']
        else:
            marker_color = colors['success']
        
        # Додаємо маркер
        fig.add_trace(go.Scattergeo(
            lon=[lon],
            lat=[lat],
            text=situation['location'],
            mode='markers',
            marker=dict(
                size=10,
                color=marker_color,
                line=dict(width=1, color='white')
            ),
            hoverinfo='text',
            hovertext=f"<b>{situation['location']}</b><br>" +
                      f"Статус: {situation['status']}<br>" +
                      f"Рівень загрози: {situation['threat_level']}/10<br>" +
                      f"Оновлено: {situation['timestamp'].split('T')[0]}"
        ))
    
    # Налаштовуємо вигляд карти
    fig.update_geos(
        visible=False,
        resolution=50,
        scope="europe",
        showcountries=True,
        countrycolor="gray",
        showsubunits=True,
        subunitcolor="blue",
        center=dict(lon=31.1656, lat=48.3794),  # Центр України
        projection_scale=5
    )
    
    fig.update_layout(
        title={
            'text': 'Карта оперативної обстановки',
            'y': 0.95,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': {'size': 20, 'color': colors['text']}
        },
        margin={"r":0, "t":30, "l":0, "b":0},
        height=500,
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        geo=dict(
            bgcolor='rgba(0,0,0,0)'
        )
    )
    
    return fig

def create_threat_analysis():
    # Отримуємо дані про рівень загрози
    threat_data = get_threat_level_history()
    
    # Створюємо графік
    fig = go.Figure()
    
    # Додаємо лінію рівня загрози
    fig.add_trace(go.Scatter(
        x=threat_data['dates'],
        y=threat_data['threat_levels'],
        mode='lines+markers',
        name='Рівень загрози',
        line=dict(color=colors['accent'], width=3),
        marker=dict(size=8, color=colors['accent']),
        fill='tozeroy',
        fillcolor=f"rgba({int(colors['accent'][1:3], 16)}, {int(colors['accent'][3:5], 16)}, {int(colors['accent'][5:7], 16)}, 0.2)"
    ))
    
    # Додаємо зони рівнів загрози
    fig.add_shape(
        type="rect",
        x0=0,
        y0=70,
        x1=len(threat_data['dates']),
        y1=100,
        fillcolor="rgba(76, 175, 80, 0.1)",
        line=dict(width=0),
        layer="below"
    )
    
    fig.add_shape(
        type="rect",
        x0=0,
        y0=40,
        x1=len(threat_data['dates']),
        y1=70,
        fillcolor="rgba(255, 152, 0, 0.1)",
        line=dict(width=0),
        layer="below"
    )
    
    fig.add_shape(
        type="rect",
        x0=0,
        y0=0,
        x1=len(threat_data['dates']),
        y1=40,
        fillcolor="rgba(244, 67, 54, 0.1)",
        line=dict(width=0),
        layer="below"
    )
    
    # Додаємо анотації для зон
    fig.add_annotation(
        x=len(threat_data['dates']) - 1,
        y=85,
        text="Низький рівень загрози",
        showarrow=False,
        font=dict(size=10, color="rgba(76, 175, 80, 0.7)")
    )
    
    fig.add_annotation(
        x=len(threat_data['dates']) - 1,
        y=55,
        text="Середній рівень загрози",
        showarrow=False,
        font=dict(size=10, color="rgba(255, 152, 0, 0.7)")
    )
    
    fig.add_annotation(
        x=len(threat_data['dates']) - 1,
        y=20,
        text="Високий рівень загрози",
        showarrow=False,
        font=dict(size=10, color="rgba(244, 67, 54, 0.7)")
    )
    
    # Налаштовуємо вигляд графіка
    fig.update_layout(
        title={
            'text': 'Динаміка рівня загрози',
            'y': 0.95,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': {'size': 20, 'color': colors['text']}
        },
        xaxis=dict(
            title='Дата',
            tickangle=45,
            tickmode='array',
            tickvals=[i for i in range(0, len(threat_data['dates']), 5)],
            ticktext=[threat_data['dates'][i] for i in range(0, len(threat_data['dates']), 5)],
            gridcolor='rgba(230, 230, 230, 0.5)'
        ),
        yaxis=dict(
            title='Рівень загрози',
            range=[0, 100],
            gridcolor='rgba(230, 230, 230, 0.5)'
        ),
        margin=dict(l=40, r=40, t=50, b=40),
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        hovermode='x unified'
    )
    
    return fig

def create_resource_management():
    # Отримуємо дані про ресурси
    resource_data = get_resources_distribution()
    
    # Створюємо дані для графіка
    resource_types = list(resource_data['by_type'].keys())
    resource_counts = [data['count'] for data in resource_data['by_type'].values()]
    resource_percentages = [data['percentage'] for data in resource_data['by_type'].values()]
    
    # Створюємо графік розподілу ресурсів за типами
    fig = go.Figure()
    
    # Додаємо стовпчики для кожного типу ресурсів
    fig.add_trace(go.Bar(
        x=resource_types,
        y=resource_counts,
        marker_color=colors['accent'],
        text=resource_percentages,
        texttemplate='%{text}%',
        textposition='outside',
        hovertemplate='<b>%{x}</b><br>Кількість: %{y}<br>Відсоток: %{text}%<extra></extra>'
    ))
    
    # Налаштовуємо вигляд графіка
    fig.update_layout(
        title={
            'text': 'Розподіл ресурсів за типами',
            'y': 0.95,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': {'size': 20, 'color': colors['text']}
        },
        xaxis=dict(
            title='Тип ресурсу',
            tickangle=45,
            gridcolor='rgba(230, 230, 230, 0.5)'
        ),
        yaxis=dict(
            title='Кількість',
            gridcolor='rgba(230, 230, 230, 0.5)'
        ),
        margin=dict(l=40, r=40, t=50, b=40),
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)'
    )
    
    return fig

def create_resource_status_chart():
    # Отримуємо дані про ресурси
    resource_data = get_resources_distribution()
    
    # Створюємо дані для графіка
    status_labels = list(resource_data['by_status'].keys())
    status_values = [data['percentage'] for data in resource_data['by_status'].values()]
    
    # Визначаємо кольори для статусів
    status_colors = {
        'Доступний': colors['success'],
        'В ремонті': colors['warning'],
        'Недоступний': colors['danger']
    }
    
    # Створюємо графік розподілу ресурсів за статусами
    fig = go.Figure()
    
    # Додаємо кругову діаграму
    fig.add_trace(go.Pie(
        labels=status_labels,
        values=status_values,
        marker=dict(
            colors=[status_colors.get(status, colors['accent']) for status in status_labels],
            line=dict(color='white', width=2)
        ),
        textinfo='label+percent',
        insidetextorientation='radial',
        hole=0.4
    ))
    
    # Налаштовуємо вигляд графіка
    fig.update_layout(
        title={
            'text': 'Статус ресурсів',
            'y': 0.95,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': {'size': 20, 'color': colors['text']}
        },
        margin=dict(l=20, r=20, t=50, b=20),
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=-0.1,
            xanchor="center",
            x=0.5
        )
    )
    
    return fig

def create_incidents_chart():
    # Отримуємо дані про інциденти
    incidents = get_recent_incidents()
    
    # Групуємо інциденти за типами
    incident_types = {}
    for incident in incidents:
        incident_type = incident['type']
        if incident_type in incident_types:
            incident_types[incident_type] += 1
        else:
            incident_types[incident_type] = 1
    
    # Створюємо дані для графіка
    types = list(incident_types.keys())
    counts = list(incident_types.values())
    
    # Створюємо графік інцидентів
    fig = go.Figure()
    
    # Додаємо стовпчики для кожного типу інцидентів
    fig.add_trace(go.Bar(
        x=types,
        y=counts,
        marker_color=colors['warning'],
        text=counts,
        textposition='outside',
        hovertemplate='<b>%{x}</b><br>Кількість: %{y}<extra></extra>'
    ))
    
    # Налаштовуємо вигляд графіка
    fig.update_layout(
        title={
            'text': 'Розподіл інцидентів за типами',
            'y': 0.95,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': {'size': 20, 'color': colors['text']}
        },
        xaxis=dict(
            title='Тип інциденту',
            tickangle=45,
            gridcolor='rgba(230, 230, 230, 0.5)'
        ),
        yaxis=dict(
            title='Кількість',
            gridcolor='rgba(230, 230, 230, 0.5)'
        ),
        margin=dict(l=40, r=40, t=50, b=40),
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)'
    )
    
    return fig

# Макет додатку
app.layout = html.Div(style={'backgroundColor': colors['background'], 'minHeight': '100vh'}, children=[
    # Заголовок
    html.Div(className="container-fluid", children=[
        html.Div(className="row mb-4", children=[
            html.Div(className="col-12", children=[
                html.H1("ІНФОРМАЦІЙНО-АНАЛІТИЧНА СИСТЕМА НГУ", className="display-5 fw-bold text-center mt-4 animate__animated animate__fadeIn"),
                html.P("Інтерактивний дашборд для моніторингу та управління ресурсами Національної гвардії України", 
                       className="lead text-muted text-center animate__animated animate__fadeIn")
            ])
        ])
    ]),
    
    # Статистика
    html.Div(className="container-fluid", children=[
        html.Div(className="row g-4 mb-4", children=[
            # Кількість підрозділів
            html.Div(className="col-md-6 col-lg-3", children=[
                html.Div(className="card h-100 animate__animated animate__fadeInUp", children=[
                    html.Div(className="card-body text-center", children=[
                        html.Div(className="d-flex align-items-center justify-content-center mb-3", children=[
                            html.I(className="fas fa-users fa-3x text-primary me-3"),
                            html.H2(id="units-count", className="display-4 mb-0")
                        ]),
                        html.H3("Підрозділи", className="card-title h5 mb-2"),
                        html.P("Загальна кількість підрозділів НГУ", className="card-text text-muted")
                    ])
                ])
            ]),
            
            # Кількість ресурсів
            html.Div(className="col-md-6 col-lg-3", children=[
                html.Div(className="card h-100 animate__animated animate__fadeInUp", 
                         style={"animation-delay": "0.1s"}, children=[
                    html.Div(className="card-body text-center", children=[
                        html.Div(className="d-flex align-items-center justify-content-center mb-3", children=[
                            html.I(className="fas fa-boxes fa-3x text-success me-3"),
                            html.H2(id="resources-count", className="display-4 mb-0")
                        ]),
                        html.H3("Ресурси", className="card-title h5 mb-2"),
                        html.P("Доступні ресурси в системі", className="card-text text-muted")
                    ])
                ])
            ]),
            
            # Кількість інцидентів
            html.Div(className="col-md-6 col-lg-3", children=[
                html.Div(className="card h-100 animate__animated animate__fadeInUp", 
                         style={"animation-delay": "0.2s"}, children=[
                    html.Div(className="card-body text-center", children=[
                        html.Div(className="d-flex align-items-center justify-content-center mb-3", children=[
                            html.I(className="fas fa-exclamation-triangle fa-3x text-warning me-3"),
                            html.H2(id="incidents-count", className="display-4 mb-0")
                        ]),
                        html.H3("Інциденти", className="card-title h5 mb-2"),
                        html.P("Зареєстровані інциденти за 30 днів", className="card-text text-muted")
                    ])
                ])
            ]),
            
            # Критичні ситуації
            html.Div(className="col-md-6 col-lg-3", children=[
                html.Div(className="card h-100 animate__animated animate__fadeInUp", 
                         style={"animation-delay": "0.3s"}, children=[
                    html.Div(className="card-body text-center", children=[
                        html.Div(className="d-flex align-items-center justify-content-center mb-3", children=[
                            html.I(className="fas fa-radiation fa-3x text-danger me-3"),
                            html.H2(id="critical-count", className="display-4 mb-0")
                        ]),
                        html.H3("Критичні ситуації", className="card-title h5 mb-2"),
                        html.P("Активні критичні ситуації", className="card-text text-muted")
                    ])
                ])
            ])
        ])
    ]),
    
    # Основні графіки
    html.Div(className="container-fluid", children=[
        html.Div(className="row g-4 mb-4", children=[
            # Карта України
            html.Div(className="col-lg-8", children=[
                html.Div(className="card h-100 animate__animated animate__fadeIn", 
                         style={"animation-delay": "0.4s"}, children=[
                    html.Div(className="card-header d-flex justify-content-between align-items-center", children=[
                        html.H5("Карта оперативної обстановки", className="mb-0"),
                        html.Button("Оновити", id="update-map", className="btn btn-sm btn-outline-primary")
                    ]),
                    html.Div(className="card-body p-0", children=[
                        dcc.Graph(id="ukraine-map", figure=create_ukraine_map(), config={'displayModeBar': False})
                    ])
                ])
            ]),
            
            # Аналіз загроз
            html.Div(className="col-lg-4", children=[
                html.Div(className="card h-100 animate__animated animate__fadeIn", 
                         style={"animation-delay": "0.5s"}, children=[
                    html.Div(className="card-header", children=[
                        html.H5("Аналіз загроз", className="mb-0")
                    ]),
                    html.Div(className="card-body", children=[
                        dcc.Graph(id="threat-analysis", figure=create_threat_analysis(), config={'displayModeBar': False})
                    ])
                ])
            ])
        ]),
        
        # Другий ряд графіків
        html.Div(className="row g-4 mb-4", children=[
            # Управління ресурсами
            html.Div(className="col-lg-6", children=[
                html.Div(className="card h-100 animate__animated animate__fadeIn", 
                         style={"animation-delay": "0.6s"}, children=[
                    html.Div(className="card-header d-flex justify-content-between align-items-center", children=[
                        html.H5("Управління ресурсами", className="mb-0"),
                        html.Div(className="btn-group", children=[
                            html.Button("За типами", id="resource-type-btn", className="btn btn-sm btn-outline-primary active"),
                            html.Button("За статусом", id="resource-status-btn", className="btn btn-sm btn-outline-primary")
                        ])
                    ]),
                    html.Div(className="card-body", children=[
                        html.Div(id="resource-chart-container", children=[
                            dcc.Graph(id="resource-chart", figure=create_resource_management(), config={'displayModeBar': False})
                        ])
                    ])
                ])
            ]),
            
            # Інциденти
            html.Div(className="col-lg-6", children=[
                html.Div(className="card h-100 animate__animated animate__fadeIn", 
                         style={"animation-delay": "0.7s"}, children=[
                    html.Div(className="card-header", children=[
                        html.H5("Останні інциденти", className="mb-0")
                    ]),
                    html.Div(className="card-body", children=[
                        html.Div(id="incidents-table-container", style={"maxHeight": "400px", "overflow": "auto"}, children=[
                            html.Table(className="table table-hover", children=[
                                html.Thead(className="table-light", children=[
                                    html.Tr(children=[
                                        html.Th("Тип"),
                                        html.Th("Локація"),
                                        html.Th("Дата"),
                                        html.Th("Статус"),
                                        html.Th("Рівень")
                                    ])
                                ]),
                                html.Tbody(id="incidents-table-body")
                            ])
                        ])
                    ])
                ])
            ])
        ]),
        
        # Третій ряд - AI прогнози
        html.Div(className="row g-4 mb-4", children=[
            html.Div(className="col-12", children=[
                html.Div(className="card animate__animated animate__fadeIn", 
                         style={"animation-delay": "0.8s"}, children=[
                    html.Div(className="card-header d-flex justify-content-between align-items-center", children=[
                        html.H5("Прогнози штучного інтелекту", className="mb-0"),
                        html.Span(className="badge bg-primary", children=["AI"])
                    ]),
                    html.Div(className="card-body", children=[
                        html.Div(className="row", id="ai-predictions-container")
                    ])
                ])
            ])
        ])
    ]),
    
    # Інтервал для автоматичного оновлення даних
    dcc.Interval(
        id='interval-component',
        interval=60*1000,  # оновлення кожну хвилину
        n_intervals=0
    )
])

# Колбеки для оновлення даних
@app.callback(
    [
        Output("units-count", "children"),
        Output("resources-count", "children"),
        Output("incidents-count", "children"),
        Output("critical-count", "children")
    ],
    [Input("interval-component", "n_intervals")]
)
def update_statistics(n):
    """Оновлення статистичних даних"""
    stats = get_dashboard_statistics()
    return [
        stats['units_count'],
        stats['resources_count'],
        stats['incidents_count'],
        stats['critical_situations_count']
    ]

@app.callback(
    Output("ukraine-map", "figure"),
    [Input("update-map", "n_clicks"), Input("interval-component", "n_intervals")]
)
def update_map(n_clicks, n_intervals):
    """Оновлення карти"""
    return create_ukraine_map()

@app.callback(
    Output("threat-analysis", "figure"),
    [Input("interval-component", "n_intervals")]
)
def update_threat_analysis(n):
    """Оновлення аналізу загроз"""
    return create_threat_analysis()

@app.callback(
    Output("resource-chart-container", "children"),
    [Input("resource-type-btn", "n_clicks"), Input("resource-status-btn", "n_clicks")]
)
def update_resource_chart(type_clicks, status_clicks):
    """Перемикання між різними видами графіків ресурсів"""
    ctx = dash.callback_context
    if not ctx.triggered:
        # За замовчуванням показуємо розподіл за типами
        return [dcc.Graph(id="resource-chart", figure=create_resource_management(), config={'displayModeBar': False})]
    else:
        button_id = ctx.triggered[0]['prop_id'].split('.')[0]
        if button_id == "resource-status-btn":
            return [dcc.Graph(id="resource-chart", figure=create_resource_status_chart(), config={'displayModeBar': False})]
        else:
            return [dcc.Graph(id="resource-chart", figure=create_resource_management(), config={'displayModeBar': False})]

@app.callback(
    Output("incidents-table-body", "children"),
    [Input("interval-component", "n_intervals")]
)
def update_incidents_table(n):
    """Оновлення таблиці інцидентів"""
    incidents = get_recent_incidents()
    rows = []
    for incident in incidents:
        # Визначаємо клас для рядка залежно від рівня важливості
        severity_class = ""
        if incident['severity'] == 'Високий':
            severity_class = "table-danger"
        elif incident['severity'] == 'Середній':
            severity_class = "table-warning"
        
        # Форматуємо дату
        try:
            date_obj = datetime.fromisoformat(incident['timestamp'])
            formatted_date = date_obj.strftime("%d.%m.%Y %H:%M")
        except:
            formatted_date = incident['timestamp']
        
        # Додаємо рядок до таблиці
        rows.append(html.Tr(className=severity_class, children=[
            html.Td(f"{incident['type']} - {incident['subtype']}"),
            html.Td(incident['location']),
            html.Td(formatted_date),
            html.Td(incident['status']),
            html.Td(incident['severity'])
        ]))
    
    return rows

@app.callback(
    Output("ai-predictions-container", "children"),
    [Input("interval-component", "n_intervals")]
)
def update_ai_predictions(n):
    """Оновлення прогнозів AI"""
    predictions = get_ai_predictions()
    prediction_cards = []
    
    for prediction in predictions:
        # Визначаємо колір картки залежно від значення прогнозу
        if prediction['value'] > 7:
            card_class = "border-danger"
            header_class = "bg-danger text-white"
        elif prediction['value'] > 4:
            card_class = "border-warning"
            header_class = "bg-warning"
        else:
            card_class = "border-success"
            header_class = "bg-success text-white"
        
        # Форматуємо дату
        try:
            date_obj = datetime.fromisoformat(prediction['timestamp'])
            formatted_date = date_obj.strftime("%d.%m.%Y")
        except:
            formatted_date = prediction['timestamp']
        
        # Створюємо картку прогнозу
        prediction_cards.append(html.Div(className="col-md-4 mb-3", children=[
            html.Div(className=f"card h-100 {card_class}", children=[
                html.Div(className=f"card-header {header_class}", children=[
                    html.H6(prediction['type'], className="mb-0")
                ]),
                html.Div(className="card-body", children=[
                    html.H5(prediction['region'], className="card-title"),
                    html.P(prediction['description'], className="card-text"),
                    html.Div(className="d-flex justify-content-between align-items-center", children=[
                        html.Span(f"Значення: {prediction['value']}"),
                        html.Span(f"Достовірність: {int(prediction['confidence']*100)}%")
                    ])
                ]),
                html.Div(className="card-footer text-muted", children=[
                    f"Оновлено: {formatted_date}"
                ])
            ])
        ]))
    
    return prediction_cards

# НЕ ЗАПУСКАЄМО app.run_server() тут, оскільки інтеграція здійснюється через DispatcherMiddleware у Flask
if __name__ == '__main__':
    app.run_server(debug=True, port=8050)
    print("Сервер запущено. Відкрийте http://127.0.0.1:8050/ у вашому браузері.")